In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from collections import deque 
import heapq
import math
import copy
import time

In [ ]:
class Cell:
    def __init__(self):
        self.is_blocked = True
        self.is_crew = False
        self.is_Alien = False
        self.is_bot = False
        self.prob_crew = 0.0
        self.prob_alien = 0.0
        self.prob_crew = 0.0

In [ ]:
class place_ship_crew :
    def __init__(self, grid, alien_detection_range , num_aliens, num_crew):
        self.grid = grid
        self.k = alien_detection_range
        self.num_aliens = num_aliens
        self.num_crew = num_crew
        self.grid_size = len(self.grid)
        self.bot_location = None
        self.crew_locations = []
        self.alien_locations = []

    # Place the bot in a random cell
    def place_bot(self):
        while self.bot_location is None:
            x = random.randint(0,self.grid_size-1)
            y = random.randint(0,self.grid_size-1)
            # Place only on the open cells
            if self.grid[x][y].is_blocked == False:
                self.bot_location = (x, y)
                self.grid[x][y].is_bot = True

    # Generate all the aliens
    def place_aliens(self):
        # Place all the aliens
        # Calculate coordinates of the bot's detection square
        detection_square = []
        for i in range(-self.k, self.k + 1):
            for j in range(-self.k, self.k + 1):
                x, y = self.bot_location[0] + i, self.bot_location[1] + j
                if 0 <= x < self.grid_size and 0 <= y < self.grid_size:
                    detection_square.append((x, y))
        # Place the give number of aliens
        while self.num_aliens>0:
            x = random.randint(0,self.grid_size-1)
            y = random.randint(0,self.grid_size-1)
            # Place only on the open cells
            if not self.grid[x][y].is_blocked and not self.grid[x][y].is_Alien and (x,y) not in detection_square:
                self.alien_locations.append((x, y))
                self.grid[x][y].is_Alien = True
                self.num_aliens-=1

    def place_crew(self):
        while self.num_crew>0:
            x = random.randint(0,self.grid_size-1)
            y = random.randint(0,self.grid_size-1)
            # Place only on the open cell
            if not self.grid[x][y].is_blocked and not self.grid[x][y].is_bot and not self.grid[x][y].is_crew:
                self.crew_locations.append((x,y))
                self.grid[x][y].is_crew = True
                self.num_crew -= 1

    # Place all the crew members
    def place_ship_members(self):
        # Place the bot, all aliens, and then captain
        self.place_bot()
        self.place_aliens()
        self.place_crew()
        return self.grid, self.bot_location, self.alien_locations, self.crew_locations

In [ ]:
class Ship_Layout:
    def __init__(self, size):
        self.size = size
        self.grid = [[Cell() for _ in range(size)] for _ in range(size)]
        self.open_cells = []
        self.open_neighbors = {}
        self.bot_location = None
        self.crew_locations = None
        self.all_alien_locations = None

    def check_for_open_neighbours(self, x, y, cell_to_be_opened, condition):
    # Initialize the open neighbors to be 0
        open_neighbors = 0
        for nx, ny in [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]:
            # Check if the neighboring coordinate is within the grid boundaries
            if 0 <= nx < len(self.grid) and 0 <= ny < len(self.grid[0]):
                # Check if the neighboring cell is an open space
                if self.grid[nx][ny].is_blocked == condition:
                    # Increment the count of neighboring empty spaces
                    open_neighbors += 1

        # Check if the count of neighboring empty spaces is exactly 1
        if open_neighbors == 1:
            cell_to_be_opened.append((x, y))

        return cell_to_be_opened

    # Identify all the dead ends
    def find_dead_ends(self):
        Dx = len(self.grid)
        Dy = len(self.grid[0])
        # Check for dead ends with exactly 1 open neightbour
        dead_ends = []
        for i in range(Dx):
            for j in range(Dy):
                dead_ends = self.check_for_open_neighbours(i, j,  [], False)

        # Open approximately half of the dead ends
        num_to_open = len(dead_ends) // 2
        for _ in range(num_to_open):
            # Pick one random cell
            x, y = random.choice(dead_ends)
            closed_neighbors = self.check_for_open_neighbours(x, y,  [], True)
            if closed_neighbors:
                nx, ny = random.choice(closed_neighbors)
                self.grid[nx][ny].is_blocked = False

    def create_ship(self):
        cell_to_open_x = np.random.randint(0,self.size-1)
        cell_to_open_y = np.random.randint(0,self.size-1)
        self.grid[cell_to_open_x][cell_to_open_y].is_blocked = False
        while True:
            next_open_cells = []
            for i in range(self.size):
                for j in range(self.size):
                    if self.grid[i][j].is_blocked==True:
                        # Check for all open neighbors for the blocked cell
                        # Append the one only with one open neighbor
                        next_open_cells = self.check_for_open_neighbours(i,j, next_open_cells, False)

            # No more cells to open
            if not next_open_cells:
                break
            else:
            # Randomly choose a cell from next_open_cells to open
                cell_to_open = random.choice(next_open_cells)
                self.grid[cell_to_open[0]][cell_to_open[1]].is_blocked = False

        # Now open all the possible dead cells
        self.find_dead_ends()

    def get_open_neighbors(self):
        for i in range(self.size):
            for j in range(self.size):
                key = (i,j)
                open_neighbors_for_key = []
                directions = [(-1,0),(1,0),(0,1),(0,-1)]
                for d in directions:
                    dx = i+d[0]
                    dy = j+d[1]
                    if 0<=dx<self.size and 0<=dy<self.size and not self.grid[dx][dy].is_blocked:
                        open_neighbors_for_key.append((dx,dy))
                self.open_neighbors[key] = open_neighbors_for_key
    
    def get_open_cells(self):
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked:
                    self.open_cells.append((i,j))

    def create_ship_and_place_crew(self, num_aliens, num_crew, alien_detection_range):
         # Generate the ship grid
        # self.create_ship()
        # Place all the crew members - bot, aliens, captain
        ship_crew_members = place_ship_crew(self.grid, alien_detection_range, num_aliens, num_crew)
        self.grid, self.bot_location, self.all_alien_locations, self.crew_locations = ship_crew_members.place_ship_members()
        self.get_open_cells()
        self.get_open_neighbors()

In [ ]:
class Bot1:
    def __init__(self, D, k, alpha, ship):
        self.size = D
        self.ship = ship
        self.k = k
        self.grid = self.ship.grid
        self.bot_location = self.ship.bot_location
        self.all_alien_locations = self.ship.all_alien_locations
        self.crew_locations = self.ship.crew_locations
        self.alpha = alpha
        self.pos_with_high_alien_beliefs = []
        self.open_cells = self.ship.open_cells
        self.open_neighbors = self.ship.open_neighbors

    def Initialize_crew_beliefs(self):
        n = len(self.open_cells)
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked and not self.grid[i][j].is_bot:
                    self.grid[i][j].prob_crew = 1/(n-1)

    def Initialize_alien_beliefs(self):
        k = 0
        # check for number of open cells in detection square
        for i in range(self.size):
            for j in range(self.size):
                if self.in_detection_square((i,j)) and not self.grid[i][j].is_blocked:
                    k +=1
        n = len(self.open_cells)-k
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked and not self.in_detection_square((i,j)):
                    self.grid[i][j].prob_alien = 1/(n)

    def check_beep_for_crew(self, crew_position):
        # Calculate Manhattan distance between bot and crew member
        distance = abs(self.bot_location[0] - crew_position[0]) + abs(self.bot_location[1] - crew_position[1])
        # Calculate probability of hearing the beep
        beep_prob = np.exp(-self.alpha * (distance - 1))
        rand_num = random.random()
        if rand_num <= beep_prob:
            return True  # Bot hears the beep
        else:
            return False  # Bot does not hear the beep

    def check_for_alien_beep(self):
        for i in range(-self.k, self.k+1):
            for j in range(-self.k, self.k+1):
                x = self.bot_location[0]+i
                y = self.bot_location[1]+j
                if 0<=x<self.size and 0<=y<self.size and self.grid[x][y].is_Alien:
                    return True
        return False

    def get_distance(self, x):
        return  abs(self.bot_location[0] - x[0]) + abs(self.bot_location[1] - x[1])

    # Update bot's knowledge at each timestep
    def update_knowledge_crew(self, Crew_beep):
        total_prob_crew = 0
        # go through each cell and update the probability of crew being present based on the beeps recieved
        for i in range(self.size):
            for j in range(self.size):
                if self.grid[i][j].is_bot:
                    self.grid[i][j].prob_crew = 0.0
                elif not self.grid[i][j].is_blocked:
                    distance = self.get_distance((i,j))
                    prob = np.exp(-self.alpha * (distance - 1))
                    # update belief of crew if beef recieved
                    if Crew_beep:
                        self.grid[i][j].prob_crew *= prob
                    else:
                        self.grid[i][j].prob_crew *= (1 - prob)
                total_prob_crew += self.grid[i][j].prob_crew

        # Normalizing
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked:
                    self.grid[i][j].prob_crew /= total_prob_crew

    def in_detection_square(self, x):
        return abs(x[0] - self.bot_location[0]) <= self.k and abs(x[1] - self.bot_location[1]) <= self.k

    def update_knowledge_alien(self, Alien_detect):
        alien_prob = np.zeros((self.size, self.size), dtype=float)
        total_prob_alien = 0
        # if we have detected the alien that means bot is inside the detection square 
        # only probabilities of cells inside the detection square needs to be updated
        if Alien_detect:
            for i in range(self.size):
                for j in range(self.size):
                    if not self.grid[i][j].is_blocked and self.in_detection_square((i,j)) and not self.grid[i][j].is_bot:
                      # alien is inside the detection square that means it either came from outside or it was inside in previous timestamp. 
                        #in both cases we need to update the cells probability based on the probability of its neighbor as alien moved from its immediate neighbors that's why we heard the detection beep 
                        neighbors = self.open_neighbors.get((i,j))
                        for n in neighbors:
                          # for all neighbors we need to have their transitional probability as well that is with the probability they moved to that particular cell
                            valid_neighbors = self.open_neighbors.get((n[0],n[1]))
                            total_neighbors = len(valid_neighbors)
                            if total_neighbors > 0:
                                alien_prob[i][j] += self.grid[n[0]][n[1]].prob_alien/(total_neighbors+1)
                        total_prob_alien += alien_prob[i][j]

        else:
            # No alien beep detected, update all cells to have zero probability of alien presence
            for i in range(self.size):
                for j in range(self.size):
                    if not self.grid[i][j].is_blocked and not self.in_detection_square((i,j)):
                        neighbors = self.open_neighbors.get((i,j))
                        for n in neighbors:
                          # for all neighbors we need to have their transitional probability as well that is with the probability they moved to that particular cell
                            valid_neighbors = self.open_neighbors.get((n[0],n[1]))
                            total_neighbors = len(valid_neighbors)
                            if total_neighbors > 0:
                                alien_prob[i][j] += self.grid[n[0]][n[1]].prob_alien/(total_neighbors+1)
                        total_prob_alien += alien_prob[i][j]

       # Normalize probabilities if total_prob_alien is not zero
        if total_prob_alien != 0:
            for i in range(self.size):
                for j in range(self.size):
                    if not self.grid[i][j].is_blocked:
                        self.grid[i][j].prob_alien = alien_prob[i][j]/total_prob_alien
        
        else:
             self.Initialize_alien_beliefs()  # Reset beliefs if total probability is zero


    def find_path(self, src, dest, avoid_alien = True):
        # print("started path planning from", src, "to " , dest)
        visited = set() # Dictionary to store the visited nodes
        parent = {}  # Dictionary to store parent nodes
        # Keep track of the nodes
        fringe = deque([src])

        while fringe:
            (row, col) = fringe.popleft()
            # Process the current cell until it reaches the destination
            if (row, col) == dest:
                # Reconstruct the path from dest to src
                path = []
                while (row, col) != src:
                    path.append((row, col))
                    row, col = parent[(row, col)]
                path.reverse()
                return path  # Path found

            # Enqueue for unvisited neighbors
            for dr, dc in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                new_row, new_col = row + dr, col + dc
                # Check if the new cell is within the boundaries 0 and length of Ship
                # Check if the new cell is not in visited
                # Check if the new cell is not blocked
                # check to see if we can find path with cells having 0 alien probability
                if 0 <= new_row < self.size and 0 <= new_col < self.size:
                    avoid_condition = (self.grid[new_row][new_col].prob_alien == 0.0) or not avoid_alien
                    if (new_row, new_col) not in visited and not self.grid[new_row][new_col].is_blocked and avoid_condition:
                        fringe.append((new_row, new_col))
                        visited.add((new_row, new_col))
                        parent[(new_row, new_col)] = (row, col)

        return None # If path not found

    # Movement decision
    def move_bot(self, crew_beep):
        # Initialize variables to store the maximum crew belief and the corresponding target cell
        max_crew_belief = 0.0
        target_cells = []
        avoid_alien = True
        best_neighbor = None
        min_alien_neighbor = float('inf')
        max_crew_neighbor = float('-inf')
        neighbors = self.open_neighbors.get((self.bot_location[0], self.bot_location[1]))

         # Prioritize moving towards high-probability crew member cells
        crew_probs = np.array([[cell.prob_crew for cell in row] for row in self.grid])

        # Find the maximum probability value and its indices
        max_prob = crew_probs.max()
        max_indices = np.argwhere(crew_probs == max_prob)

        # Extract the coordinates of cells with the highest probability
        target_cells = [(i, j) for i, j in max_indices]
         # Prioritize moving towards high-probability crew member cells
        for neighbor in neighbors:
            i, j = neighbor
            crew_belief = self.grid[i][j].prob_crew
            alien_belief = self.grid[i][j].prob_alien
            if crew_beep:
                if crew_belief > max_crew_neighbor:
                    max_crew_neighbor = crew_belief
                    best_neighbor = neighbor
            else:
                if alien_belief < min_alien_neighbor:
                    min_alien_neighbor = alien_belief
                    best_neighbor = neighbor

        # Choose one of the target cells randomly if there are multiple cells with max crew belief
        if target_cells:
            target_cell = random.choice(target_cells)

        # print(f"bot move to {target_cell} with probability {max_crew_belief}")
        path = self.find_path(self.bot_location, target_cell, avoid_alien)
        if path is not None and len(path)>0:
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
            # Update the bot's location
            self.bot_location = path[0]
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
        else:
            avoid_alien = False
            path = self.find_path(self.bot_location, target_cell, avoid_alien)
            if path is not None and len(path)>0:
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                # Update the bot's location
                self.bot_location = path[0]
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
            else:
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                 # Update the bot's location
                self.bot_location = best_neighbor
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True

    # Handling alien movement
    def handle_alien_movement(self):
        new_alien_positions = []
        for i in range(len(self.all_alien_locations)):
            x, y = self.all_alien_locations[i]
            directions = [(0,1),(0,-1),(-1,0),(1,0)]
            random.shuffle(directions)
            possible_moves = [
            (nx, ny)
            for dx, dy in directions
            if 0 <= (nx := x + dx) < self.size and 0 <= (ny := y + dy) < self.size and not self.grid[nx][ny].is_blocked and not self.grid[nx][ny].is_Alien
            ]
            if(possible_moves):
                new_position = random.choice(possible_moves)
                new_alien_positions.append(new_position)
                ax, ay = new_position
                self.grid[ax][ay].is_Alien = True
                self.grid[x][y].is_Alien = False

        self.all_alien_locations = new_alien_positions
        # Adjust movement strategy to avoid alien

    # Main game loop
    def bot_function(self):
        steps = 1000
        self.Initialize_crew_beliefs()
        self.Initialize_alien_beliefs()
        for step in range(steps):
            # print("self.crew_locations self.bot_location self.alien_locations", self.crew_locations, self.bot_location, self.all_alien_locations)
            Crew_beep = self.check_beep_for_crew(self.crew_locations[0])
            self.update_knowledge_crew(Crew_beep)
            Alien_detect = self.check_for_alien_beep()
            self.update_knowledge_alien(Alien_detect)

            self.move_bot(Crew_beep)
            # Check if the bot reaches the crew
            if self.bot_location in self.crew_locations:
                # print("Congratulations! Bot reached the crew in steps:", step)
                return True, step, 1
            # Check if the bot is caught by an alien
            if self.bot_location in self.all_alien_locations:
                # print("Game over! Bot caught by an alien in steps:", step)
                return False, step, 0
            Alien_detect = self.check_for_alien_beep()
            self.update_knowledge_alien(Alien_detect)
            self.handle_alien_movement()
            # print(self.all_alien_locations)
            # Check if the bot is caught by an alien
            if self.bot_location in self.all_alien_locations:
                # print("Game over! Bot caught by an alien in steps:", step)
                return False, step, 0

            # visualize_grid(self.grid)
        # print("Game over! Bot couldn't reach the crew in steps:", step)
        return False, step, 0

In [ ]:
class Bot2:
    def __init__(self, D, k, alpha, ship):
        self.size = D
        self.ship = ship
        self.k = k
        self.grid = self.ship.grid
        self.bot_location = self.ship.bot_location
        self.all_alien_locations = self.ship.all_alien_locations
        self.crew_locations = self.ship.crew_locations
        self.alpha = alpha
        self.pos_with_high_alien_beliefs = []
        self.open_cells = self.ship.open_cells
        self.open_neighbors = self.ship.open_neighbors
        self.alien_threshold = 0.1

    def Initialize_crew_beliefs(self):
        n = len(self.open_cells)
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked and not self.grid[i][j].is_bot:
                    self.grid[i][j].prob_crew = 1/(n-1)

    def Initialize_alien_beliefs(self):
        k = 0
        # check for number of open cells in detection square
        for i in range(self.size):
            for j in range(self.size):
                if self.in_detection_square((i,j)) and not self.grid[i][j].is_blocked:
                    k +=1
        n = len(self.open_cells)-k
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked and not self.in_detection_square((i,j)):
                    self.grid[i][j].prob_alien = 1/(n)

    def check_beep_for_crew(self, crew_position):
        # Calculate Manhattan distance between bot and crew member
        distance = abs(self.bot_location[0] - crew_position[0]) + abs(self.bot_location[1] - crew_position[1])
        # Calculate probability of hearing the beep
        beep_prob = np.exp(-self.alpha * (distance - 1))
        rand_num = random.random()
        if rand_num <= beep_prob:
            return True  # Bot hears the beep
        else:
            return False  # Bot does not hear the beep

    def check_for_alien_beep(self):
        for i in range(-self.k, self.k+1):
            for j in range(-self.k, self.k+1):
                x = self.bot_location[0]+i
                y = self.bot_location[1]+j
                if 0<=x<self.size and 0<=y<self.size and self.grid[x][y].is_Alien:
                    return True
        return False

    def get_distance(self, x):
        return  abs(self.bot_location[0] - x[0]) + abs(self.bot_location[1] - x[1])

    # Update bot's knowledge at each timestep
    def update_knowledge_crew(self, Crew_beep):
        total_prob_crew = 0
        # go through each cell and update the probability of crew being present based on the beeps recieved
        for i in range(self.size):
            for j in range(self.size):
                if self.grid[i][j].is_bot:
                    self.grid[i][j].prob_crew = 0.0
                elif not self.grid[i][j].is_blocked:
                    distance = self.get_distance((i,j))
                    prob = np.exp(-self.alpha * (distance - 1))
                    # update belief of crew if beef recieved
                    if Crew_beep:
                        self.grid[i][j].prob_crew *= prob
                    else:
                        self.grid[i][j].prob_crew *= (1 - prob)
                total_prob_crew += self.grid[i][j].prob_crew

        # Normalizing
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked:
                    self.grid[i][j].prob_crew /= total_prob_crew

    #     self.update_knowledge_crew_along_distance()

    # def update_knowledge_crew_along_distance(self):
    #     total_prob_crew = 0
    #     # go through each cell and update the probability of crew being present based on the beeps recieved
    #     for i in range(self.size):
    #         for j in range(self.size):
    #             if not self.grid[i][j].is_blocked and not self.grid[i][j].is_bot :
    #                 self.grid[i][j].prob_crew *= self.grid[i][j].prob_crew * (1/self.get_distance((i,j)))

    def in_detection_square(self, x):
        return abs(x[0] - self.bot_location[0]) <= self.k and abs(x[1] - self.bot_location[1]) <= self.k

    def update_knowledge_alien(self, Alien_detect):
        alien_prob = np.zeros((self.size, self.size), dtype=float)
        total_prob_alien = 0
        # if we have detected the alien that means bot is inside the detection square 
        # only probabilities of cells inside the detection square needs to be updated
        if Alien_detect:
            for i in range(self.size):
                for j in range(self.size):
                    if not self.grid[i][j].is_blocked and self.in_detection_square((i,j)) and not self.grid[i][j].is_bot:
                      # alien is inside the detection square that means it either came from outside or it was inside in previous timestamp. 
                        #in both cases we need to update the cells probability based on the probability of its neighbor as alien moved from its immediate neighbors that's why we heard the detection beep 
                        neighbors = self.open_neighbors.get((i,j))
                        for n in neighbors:
                          # for all neighbors we need to have their transitional probability as well that is with the probability they moved to that particular cell
                            valid_neighbors = self.open_neighbors.get((n[0],n[1]))
                            total_neighbors = len(valid_neighbors)
                            if total_neighbors > 0:
                                alien_prob[i][j] += self.grid[n[0]][n[1]].prob_alien/(total_neighbors+1)
                        total_prob_alien += alien_prob[i][j]

        else:
            # No alien beep detected, update all cells to have zero probability of alien presence
            for i in range(self.size):
                for j in range(self.size):
                    if not self.grid[i][j].is_blocked and not self.in_detection_square((i,j)):
                        neighbors = self.open_neighbors.get((i,j))
                        for n in neighbors:
                          # for all neighbors we need to have their transitional probability as well that is with the probability they moved to that particular cell
                            valid_neighbors = self.open_neighbors.get((n[0],n[1]))
                            total_neighbors = len(valid_neighbors)
                            if total_neighbors > 0:
                                alien_prob[i][j] += self.grid[n[0]][n[1]].prob_alien/(total_neighbors+1)
                        total_prob_alien += alien_prob[i][j]

       # Normalize probabilities if total_prob_alien is not zero
        if total_prob_alien != 0:
            for i in range(self.size):
                for j in range(self.size):
                    if not self.grid[i][j].is_blocked:
                        self.grid[i][j].prob_alien = alien_prob[i][j]/total_prob_alien
        
        else:
             self.Initialize_alien_beliefs()  # Reset beliefs if total probability is zero
        
    #     self.update_knowledge_alien_along_distance()

    # def update_knowledge_alien_along_distance(self):
    #     total_prob_crew = 0
    #     # go through each cell and update the probability of crew being present based on the beeps recieved
    #     for i in range(self.size):
    #         for j in range(self.size):
    #             if not self.grid[i][j].is_blocked and not self.grid[i][j].is_bot :
    #                 self.grid[i][j].prob_alien_along_distance *= self.grid[i][j].prob_alien * (1/self.get_distance((i,j)))

    def find_path(self, src, dest, avoid_alien = True):
        # print("started path planning from", src, "to " , dest)
        visited = set() # Dictionary to store the visited nodes
        parent = {}  # Dictionary to store parent nodes
        # Keep track of the nodes
        fringe = deque([src])

        while fringe:
            (row, col) = fringe.popleft()
            # Process the current cell until it reaches the destination
            if (row, col) == dest:
                # Reconstruct the path from dest to src
                path = []
                while (row, col) != src:
                    path.append((row, col))
                    row, col = parent[(row, col)]
                path.reverse()
                return path  # Path found

            # Enqueue for unvisited neighbors
            for dr, dc in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                new_row, new_col = row + dr, col + dc
                # Check if the new cell is within the boundaries 0 and length of Ship
                # Check if the new cell is not in visited
                # Check if the new cell is not blocked
                # check to see if we can find path with cells having 0 alien probability
                if 0 <= new_row < self.size and 0 <= new_col < self.size:
                    avoid_condition = (self.grid[new_row][new_col].prob_alien <= self.alien_threshold) or not avoid_alien
                    if (new_row, new_col) not in visited and not self.grid[new_row][new_col].is_blocked and avoid_condition:
                        fringe.append((new_row, new_col))
                        visited.add((new_row, new_col))
                        parent[(new_row, new_col)] = (row, col)

        return None # If path not found

    # Movement decision
    def move_bot(self, crew_beep):
        # Initialize variables to store the maximum crew belief and the corresponding target cell
        max_crew_belief = 0.0
        target_cells = []
        avoid_alien = True
        best_neighbor = None
        min_alien_neighbor = float('inf')
        max_crew_neighbor = float('-inf')
        neighbors = self.open_neighbors.get((self.bot_location[0], self.bot_location[1]))

         # Prioritize moving towards high-probability crew member cells
        crew_probs = np.array([[cell.prob_crew for cell in row] for row in self.grid])

        # Find the maximum probability value and its indices
        max_prob = crew_probs.max()
        max_indices = np.argwhere(crew_probs == max_prob)

        # Extract the coordinates of cells with the highest probability
        target_cells = [(i, j) for i, j in max_indices]
         # Prioritize moving towards high-probability crew member cells
        for neighbor in neighbors:
            i, j = neighbor
            crew_belief = self.grid[i][j].prob_crew
            alien_belief = self.grid[i][j].prob_alien
            if crew_beep:
                if crew_belief > max_crew_neighbor:
                    max_crew_neighbor = crew_belief
                    best_neighbor = neighbor
            else:
                if alien_belief < min_alien_neighbor:
                    min_alien_neighbor = alien_belief
                    best_neighbor = neighbor

        # Choose one of the target cells randomly if there are multiple cells with max crew belief
        if target_cells:
            target_cell = random.choice(target_cells)

        # print(f"bot move to {target_cell} with probability {max_crew_belief}")
        path = self.find_path(self.bot_location, target_cell, avoid_alien)
        if path is not None and len(path)>0:
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
            # Update the bot's location
            self.bot_location = path[0]
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
        else:
            avoid_alien = False
            path = self.find_path(self.bot_location, target_cell, avoid_alien)
            if path is not None and len(path)>0:
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                # Update the bot's location
                self.bot_location = path[0]
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
            else:
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                 # Update the bot's location
                self.bot_location = best_neighbor
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True

    # Handling alien movement
    def handle_alien_movement(self):
        new_alien_positions = []
        for i in range(len(self.all_alien_locations)):
            x, y = self.all_alien_locations[i]
            directions = [(0,1),(0,-1),(-1,0),(1,0)]
            random.shuffle(directions)
            possible_moves = [
            (nx, ny)
            for dx, dy in directions
            if 0 <= (nx := x + dx) < self.size and 0 <= (ny := y + dy) < self.size and not self.grid[nx][ny].is_blocked and not self.grid[nx][ny].is_Alien
            ]
            if(possible_moves):
                new_position = random.choice(possible_moves)
                new_alien_positions.append(new_position)
                ax, ay = new_position
                self.grid[ax][ay].is_Alien = True
                self.grid[x][y].is_Alien = False

        self.all_alien_locations = new_alien_positions
        # Adjust movement strategy to avoid alien

    # Main game loop
    def bot_function(self):
        steps = 1000
        self.Initialize_crew_beliefs()
        self.Initialize_alien_beliefs()
        for step in range(steps):
            # print("self.crew_locations self.bot_location self.alien_locations", self.crew_locations, self.bot_location, self.all_alien_locations)
            Crew_beep = self.check_beep_for_crew(self.crew_locations[0])
            self.update_knowledge_crew(Crew_beep)
            Alien_detect = self.check_for_alien_beep()
            self.update_knowledge_alien(Alien_detect)

            self.move_bot(Crew_beep)
            # Check if the bot reaches the crew
            if self.bot_location in self.crew_locations:
                # print("Congratulations! Bot reached the crew in steps:", step)
                return True, step, 1
            # Check if the bot is caught by an alien
            if self.bot_location in self.all_alien_locations:
                # print("Game over! Bot caught by an alien in steps:", step)
                return False, step, 0
            Alien_detect = self.check_for_alien_beep()
            self.update_knowledge_alien(Alien_detect)
            self.handle_alien_movement()
            # print(self.all_alien_locations)
            # Check if the bot is caught by an alien
            if self.bot_location in self.all_alien_locations:
                # print("Game over! Bot caught by an alien in steps:", step)
                return False, step, 0

            # visualize_grid(self.grid)
        # print("Game over! Bot couldn't reach the crew in steps:", step)
        return False, step, 0

In [ ]:
class Bot3:
    def __init__(self, D, k, alpha, ship):
        self.size = D
        self.ship = ship
        self.k = k
        self.grid = self.ship.grid
        self.bot_location = self.ship.bot_location
        self.all_alien_locations = self.ship.all_alien_locations
        self.crew_locations = self.ship.crew_locations
        self.alpha = alpha
        self.pos_with_high_alien_beliefs = []
        self.alien_threshold = 0.1
        self.open_cells = self.ship.open_cells
        self.open_neighbors = self.ship.open_neighbors

    def Initialize_crew_beliefs(self):
        n = len(self.open_cells)
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked and not self.grid[i][j].is_bot:
                    self.grid[i][j].prob_crew = 1/(n-1)
    
    def Initialize_alien_beliefs(self):
        k = 0
        # check for number of open cells in detection square 
        for i in range(self.size):
            for j in range(self.size):
                if self.in_detection_square((i,j)) and not self.grid[i][j].is_blocked:
                    k +=1
        n = len(self.open_cells)-k        
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked and not self.in_detection_square((i,j)):
                    self.grid[i][j].prob_alien = 1/(n)
    
    def check_beep_for_crew(self, crew_position):
        # Calculate Manhattan distance between bot and crew member
        distance = abs(self.bot_location[0] - crew_position[0]) + abs(self.bot_location[1] - crew_position[1])
        # Calculate probability of hearing the beep
        beep_prob = np.exp(-self.alpha * (distance - 1))
        rand_num = random.random()
        if rand_num <= beep_prob:
            return True  # Bot hears the beep
        else:
            return False  # Bot does not hear the beep

    def check_for_alien_beep(self):
        for i in range(-self.k, self.k+1):
            for j in range(-self.k, self.k+1):
                x = self.bot_location[0]+i
                y = self.bot_location[1]+j
                if 0<=x<self.size and 0<=y<self.size and self.grid[x][y].is_Alien:
                    return True
        return False           

    def get_distance(self, x):
        return  abs(self.bot_location[0] - x[0]) + abs(self.bot_location[1] - x[1])
    
    def in_detection_square(self, x):
        return abs(x[0] - self.bot_location[0]) <= self.k and abs(x[1] - self.bot_location[1]) <= self.k

    # Update bot's knowledge at each timestep
    def update_knowledge_crew(self, Crew_beep):
        total_prob_crew = 0
        # go through each cell and update the probability of crew being present based on the beeps recieved
        for i in range(self.size):
            for j in range(self.size):
                if self.grid[i][j].is_bot:
                    self.grid[i][j].prob_crew = 0.0
                elif not self.grid[i][j].is_blocked:
                    distance = self.get_distance((i,j))
                    prob = np.exp(-self.alpha * (distance - 1))
                    # update belief of crew if beef recieved
                    if any(Crew_beep):
                        self.grid[i][j].prob_crew *= prob
                    else:
                        self.grid[i][j].prob_crew *= (1 - prob)
                total_prob_crew += self.grid[i][j].prob_crew
        
        # Normalizing
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked:
                    self.grid[i][j].prob_crew /= total_prob_crew
                    
    def update_knowledge_alien(self, Alien_detect):
        alien_prob = np.zeros((self.size, self.size), dtype=float)
        total_prob_alien = 0
        # if we have detected the alien that means bot is inside the detection square and only probabilities of cells inside the detection square needs to be updated
        if Alien_detect:
            for i in range(self.size):
                for j in range(self.size):
                    if not self.grid[i][j].is_blocked and self.in_detection_square((i,j)) and not self.grid[i][j].is_bot:
                      # alien is inside the detection square that means it either came from outside or it was inside in previous timestamp. 
                        #in both cases we need to update the cells probability based on the probability of its neighbor as alien moved from its immediate neighbors that's why we heard the detection beep 
                        neighbors = self.open_neighbors.get((i,j))
                        for n in neighbors:
                          # for all neighbors we need to have their transitional probability as well that is with the probability they moved to that particular cell
                            valid_neighbors = self.open_neighbors.get((n[0],n[1]))
                            total_neighbors = len(valid_neighbors)
                            if total_neighbors > 0:
                                alien_prob[i][j] += self.grid[n[0]][n[1]].prob_alien/(total_neighbors+1)
                        total_prob_alien += alien_prob[i][j]

        else:
            # No alien beep detected, update all cells to have zero probability of alien presence
            for i in range(self.size):
                for j in range(self.size):
                    if not self.grid[i][j].is_blocked and not self.in_detection_square((i,j)):
                        neighbors = self.open_neighbors.get((i,j))
                        for n in neighbors:
                          # for all neighbors we need to have their transitional probability as well that is with the probability they moved to that particular cell
                            valid_neighbors = self.open_neighbors.get((n[0],n[1]))
                            total_neighbors = len(valid_neighbors)
                            if total_neighbors > 0:
                                alien_prob[i][j] += self.grid[n[0]][n[1]].prob_alien/(total_neighbors+1)
                        total_prob_alien += alien_prob[i][j]

        
       # Normalize probabilities if total_prob_alien is not zero
        if total_prob_alien != 0:
            for i in range(self.size):
                for j in range(self.size):
                    if not self.grid[i][j].is_blocked:
                        self.grid[i][j].prob_alien = alien_prob[i][j]/total_prob_alien
        
        else:
             self.Initialize_alien_beliefs()  # Reset beliefs if total probability is zero

        alien_probabilities = [(i, j) for i in range(self.size) for j in range(self.size)]
        self.pos_with_high_alien_beliefs = heapq.nlargest(5, alien_probabilities, key=lambda x: self.grid[x[0]][x[1]].prob_alien)


    def find_path(self, src, dest, avoid_alien = True):
        # print("started path planning from", src, "to " , dest)
        visited = set() # Dictionary to store the visited nodes
        parent = {}  # Dictionary to store parent nodes
        # Keep track of the nodes
        fringe = deque([src])

        while fringe:
            (row, col) = fringe.popleft()
            # Process the current cell until it reaches the destination
            if (row, col) == dest:
                # Reconstruct the path from dest to src
                path = []
                while (row, col) != src:
                    path.append((row, col))
                    row, col = parent[(row, col)]
                path.reverse()
                return path  # Path found
            
            # Enqueue for unvisited neighbors
            for dr, dc in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                new_row, new_col = row + dr, col + dc
                # Check if the new cell is within the boundaries 0 and length of Ship
                # Check if the new cell is not in alien location and not in visited
                # Check if the new cell is not blocked 
                if 0 <= new_row < self.size and 0 <= new_col < self.size:
                    avoid_condition =  (self.grid[new_row][new_col].prob_alien < self.alien_threshold) or (not avoid_alien and (new_row, new_col) not in self.pos_with_high_alien_beliefs)
                    if (new_row, new_col) not in visited and not self.grid[new_row][new_col].is_blocked and avoid_condition:
                        fringe.append((new_row, new_col))
                        visited.add((new_row, new_col))
                        parent[(new_row, new_col)] = (row, col)

        return None # If path not found

   
    # Movement decision
    def move_bot(self, crew_beep):
        # Initialize variables to store the maximum crew belief and the corresponding target cell
        max_crew_belief = 0.0
        target_cells = []
        avoid_alien = True
        best_neighbor = None
        min_alien_neighbor = float('inf')
        max_crew_neighbor = float('-inf')
        neighbors = self.open_neighbors.get((self.bot_location[0], self.bot_location[1]))

         # Prioritize moving towards high-probability crew member cells
        crew_probs = np.array([[cell.prob_crew for cell in row] for row in self.grid])

        # Find the maximum probability value and its indices
        max_prob = crew_probs.max()
        max_indices = np.argwhere(crew_probs == max_prob)

        # Extract the coordinates of cells with the highest probability
        target_cells = [(i, j) for i, j in max_indices]
         # Prioritize moving towards high-probability crew member cells
        for neighbor in neighbors:
            i, j = neighbor
            crew_belief = self.grid[i][j].prob_crew
            alien_belief = self.grid[i][j].prob_alien
            if any(crew_beep):
                if crew_belief > max_crew_neighbor:
                    max_crew_neighbor = crew_belief
                    best_neighbor = neighbor
            else:
                if alien_belief < min_alien_neighbor:
                    min_alien_neighbor = alien_belief
                    best_neighbor = neighbor

        # Choose one of the target cells randomly if there are multiple cells with max crew belief
        if target_cells:
            target_cell = random.choice(target_cells)

        # print(f"bot move to {target_cell} with probability {max_crew_belief}")
        path = self.find_path(self.bot_location, target_cell, avoid_alien)
        if path is not None and len(path)>0:
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
            # Update the bot's location
            self.bot_location = path[0]
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
        else:
            avoid_alien = False
            path = self.find_path(self.bot_location, target_cell, avoid_alien)
            if path is not None and len(path)>0:
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                # Update the bot's location
                self.bot_location = path[0]
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
            else:
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                 # Update the bot's location
                self.bot_location = best_neighbor
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True            

    # Handling alien movement
    def handle_alien_movement(self):
        new_alien_positions = []
        for i in range(len(self.all_alien_locations)):
            x, y = self.all_alien_locations[i]
            directions = [(0,1),(0,-1),(-1,0),(1,0)]
            random.shuffle(directions)
            possible_moves = [
            (nx, ny) 
            for dx, dy in directions
            if 0 <= (nx := x + dx) < self.size and 0 <= (ny := y + dy) < self.size and not self.grid[nx][ny].is_blocked and not self.grid[nx][ny].is_Alien 
            ]
            if(possible_moves):
                new_position = random.choice(possible_moves)
                new_alien_positions.append(new_position)
                ax, ay = new_position
                self.grid[ax][ay].is_Alien = True
                self.grid[x][y].is_Alien = False
        
        self.all_alien_locations = new_alien_positions
        # Update knowledge of alien's possible locations
        # Adjust movement strategy to avoid alien

    # Main function
    def bot_function(self):
        steps = 1000
        self.Initialize_crew_beliefs()
        self.Initialize_alien_beliefs()
        num_of_crew_reached = 0
        for step in range(steps):
            # print("self.crew_locations self.bot_location self.alien_locations", self.crew_locations, self.bot_location, self.all_alien_locations)
            crew_beep = []
            for crew_location in self.crew_locations:
                crew_beep.append(self.check_beep_for_crew(crew_location))
            Alien_detect = self.check_for_alien_beep()
            self.update_knowledge_crew(crew_beep)
            self.update_knowledge_alien(Alien_detect)
            self.move_bot(crew_beep)
            # Check if the bot reaches the crew
            if self.bot_location in self.crew_locations:
                num_of_crew_reached += 1
                if num_of_crew_reached == 2:
                    # print("Congratulations! Bot reached the crew in steps:", step)
                    return True, step, num_of_crew_reached
                else:
                    # print("Congratz, CREW1 Reached")
                    self.grid[self.bot_location[0]][self.bot_location[1]].is_crew =  False 
                    self.crew_locations.remove(self.bot_location)
            
            # Check if the bot is caught by an alien
            if self.bot_location in self.all_alien_locations:
                # print("Game over! Bot caught by an alien in steps:", step)
                return False, step, num_of_crew_reached
            
            Alien_detect = self.check_for_alien_beep()
            self.update_knowledge_alien(Alien_detect)
            # visualize_grid(self.grid)
            # Check if the bot reaches the crew
            self.handle_alien_movement()

            if self.bot_location in self.all_alien_locations:
                # print("Game over! Bot caught by an alien in steps:", step)
                return False, step, num_of_crew_reached
            
            # visualize_grid(self.grid)
            
        # print("Game over! Bot couldn't reach the crew in steps:", _)
        return False, step, num_of_crew_reached


In [ ]:
class Bot4:
    def __init__(self, D, k, alpha, ship):
        self.size = D
        self.ship = ship
        self.k = k
        self.grid = self.ship.grid
        self.bot_location = self.ship.bot_location
        self.all_alien_locations = self.ship.all_alien_locations
        self.crew_locations = self.ship.crew_locations
        self.alpha = alpha
        self.crew_beliefs = {}
        self.crew_found = None
        self.pos_with_high_alien_beliefs = []
        self.open_cells = self.ship.open_cells
        self.open_neighbors = self.ship.open_neighbors
        self.alien_threshold = 0.05

    def Initialize_crew_beliefs(self):
        num_open_cells = len(self.open_cells)
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked and not self.grid[i][j].is_bot:
                    for m in range(self.size):
                        for n in range(self.size):
                            if not self.grid[m][n].is_blocked and not self.grid[i][j].is_bot and (i!=m and j!=n):
                                key = tuple(sorted(((i, j), (m, n))))  # Sort the tuple to ensure consistent order key = ((i, j), (m, n))
                                self.crew_beliefs[key] = 1/(num_open_cells-1)*1/(num_open_cells-1)

    def Initialize_alien_beliefs(self):
        k = 0
        # check for number of open cells in detection square 
        for i in range(self.size):
            for j in range(self.size):
                if self.in_detection_square((i,j)) and not self.grid[i][j].is_blocked:
                    k +=1
        n = len(self.open_cells)-k        
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked and not self.in_detection_square((i,j)):
                    self.grid[i][j].prob_alien = 1/(n)
    
    def check_beep_for_crew(self, crew_position):
        # Calculate Manhattan distance between bot and crew member
        distance = abs(self.bot_location[0] - crew_position[0]) + abs(self.bot_location[1] - crew_position[1])
        # Calculate probability of hearing the beep
        beep_prob = np.exp(-self.alpha * (distance - 1))
        rand_num = random.random()
        if rand_num <= beep_prob:
            return True  # Bot hears the beep
        else:
            return False  # Bot does not hear the beep

    def check_for_alien_beep(self):
        for i in range(-self.k, self.k+1):
            for j in range(-self.k, self.k+1):
                x = self.bot_location[0]+i
                y = self.bot_location[1]+j
                if 0<=x<self.size and 0<=y<self.size and self.grid[x][y].is_Alien:
                    return True
        return False 

    def get_distance(self, x):
        return  abs(self.bot_location[0] - x[0]) + abs(self.bot_location[1] - x[1])
    
    
    def initialize_prob(self):
        n = len(self.open_cells)
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked and not self.grid[i][j].is_bot:
                    self.grid[i][j].prob_crew = 1/(n-1)  

    # Update bot's knowledge at each timestep
    def update_knowledge_crew(self, crew_beep):
        probability_sum = 0
        if self.crew_found == None:
            for key in self.crew_beliefs:
                crew_i = key[0]
                crew_j = key[1]

                if self.bot_location in key:
                    self.crew_beliefs[(crew_i, crew_j)] = 0.0 #set belief to 0 for current bot location as we know crew member is not there  
                    continue  # Skip this iteration if bot location is part of key

                distance_to_crew_i = self.get_distance(crew_i)
                distance_to_crew_j = self.get_distance(crew_j)
                crew_i_prob = np.exp(-self.alpha * (distance_to_crew_i - 1))
                crew_j_prob = np.exp(-self.alpha * (distance_to_crew_j - 1))
                
                if any(crew_beep):
                    likelihood = crew_i_prob + crew_j_prob - crew_i_prob * crew_j_prob
                else:
                    likelihood = (1-crew_i_prob) * (1-crew_j_prob)
                            
                # Calculate prior probability P(crew_i and crew_j)
                prior_prob = self.crew_beliefs.get((crew_i, crew_j), 0.0)

                new_prob = prior_prob * likelihood
                self.crew_beliefs[(crew_i, crew_j)] = new_prob

                probability_sum += self.crew_beliefs[(crew_i, crew_j)]
            
            # Normalizing 
            normalized_beliefs = {}
            for key, value in self.crew_beliefs.items():
                normalized_beliefs[key] = value / probability_sum
    
            self.crew_beliefs = normalized_beliefs
        
        else:
        # go through each cell and update the probability of crew being present based on the beeps recieved
            for i in range(self.size):
                for j in range(self.size):
                    if self.grid[i][j].is_bot:
                        self.grid[i][j].prob_crew = 0.0
                    elif not self.grid[i][j].is_blocked:
                        distance = self.get_distance((i,j))
                        prob = np.exp(-self.alpha * (distance - 1))
                        # update belief of crew if beef recieved
                        if any(crew_beep):
                            self.grid[i][j].prob_crew *= prob
                        else:
                            self.grid[i][j].prob_crew *= (1 - prob)
                    probability_sum +=self.grid[i][j].prob_crew
            
            # Normalizing 
            if probability_sum > 0:
                for i in range(self.size):
                    for j in range(self.size):
                        if not self.grid[i][j].is_blocked:
                            self.grid[i][j].prob_crew /= probability_sum
            else:
                self.initialize_prob()
        
    
    def in_detection_square(self, x):
        return abs(x[0] - self.bot_location[0]) <= self.k and abs(x[1] - self.bot_location[1]) <= self.k

    def update_knowledge_alien(self, Alien_detect):
        alien_prob = np.zeros((self.size, self.size), dtype=float)
        total_prob_alien = 0
        # if we have detected the alien that means bot is inside the detection square and only probabilities of cells inside the detection square needs to be updated
        if Alien_detect:
            for i in range(self.size):
                for j in range(self.size):
                    if not self.grid[i][j].is_blocked and self.in_detection_square((i,j)) and not self.grid[i][j].is_bot:
                      # alien is inside the detection square that means it either came from outside or it was inside in previous timestamp. 
                        #in both cases we need to update the cells probability based on the probability of its neighbor as alien moved from its immediate neighbors that's why we heard the detection beep 
                        neighbors = self.open_neighbors.get((i,j))
                        for n in neighbors:
                          # for all neighbors we need to have their transitional probability as well that is with the probability they moved to that particular cell
                            valid_neighbors = self.open_neighbors.get((n[0],n[1]))
                            total_neighbors = len(valid_neighbors)
                            if total_neighbors > 0:
                                alien_prob[i][j] += self.grid[n[0]][n[1]].prob_alien/(total_neighbors+1)
                        total_prob_alien += alien_prob[i][j]

        else:
            # No alien beep detected, update all cells to have zero probability of alien presence
            for i in range(self.size):
                for j in range(self.size):
                    if not self.grid[i][j].is_blocked and not self.in_detection_square((i,j)):
                        neighbors = self.open_neighbors.get((i,j))
                        for n in neighbors:
                          # for all neighbors we need to have their transitional probability as well that is with the probability they moved to that particular cell
                            valid_neighbors = self.open_neighbors.get((n[0],n[1]))
                            total_neighbors = len(valid_neighbors)
                            if total_neighbors > 0:
                                alien_prob[i][j] += self.grid[n[0]][n[1]].prob_alien/(total_neighbors+1)
                        total_prob_alien += alien_prob[i][j]

        
       # Normalize probabilities if total_prob_alien is not zero
        if total_prob_alien != 0:
            for i in range(self.size):
                for j in range(self.size):
                    if not self.grid[i][j].is_blocked:
                        self.grid[i][j].prob_alien = alien_prob[i][j]/total_prob_alien
        
        else:
             self.Initialize_alien_beliefs()  # Reset beliefs if total probability is zero

        alien_probabilities = [(i, j) for i in range(self.size) for j in range(self.size)]
        sorted_alien_probabilities = sorted(alien_probabilities, key=lambda x: self.grid[x[0]][x[1]].prob_alien, reverse=True)
        self.pos_with_high_alien_beliefs = sorted_alien_probabilities[:5]

    
    def find_path(self, src, dest, avoid_alien = True):
        # print("started path planning from", src, "to " , dest)
        visited = set() # Dictionary to store the visited nodes
        parent = {}  # Dictionary to store parent nodes
        # Keep track of the nodes
        fringe = deque([src])
        
        while fringe:
            (row, col) = fringe.popleft()
            # Process the current cell until it reaches the destination
            if (row, col) == dest:
                # Reconstruct the path from dest to src
                path = []
                while (row, col) != src:
                    path.append((row, col))
                    row, col = parent[(row, col)]
                path.reverse()
                return path  # Path found
            
            # Enqueue for unvisited neighbors
            for dr, dc in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                new_row, new_col = row + dr, col + dc
                # Check if the new cell is within the boundaries 0 and length of Ship
                # Check if the new cell is not in alien location and not in visited
                # Check if the new cell is not blocked 
                if 0 <= new_row < self.size and 0 <= new_col < self.size:
                    avoid_condition =  (self.grid[new_row][new_col].prob_alien < self.alien_threshold) or (not avoid_alien and (new_row, new_col) not in self.pos_with_high_alien_beliefs)
                    if (new_row, new_col) not in visited and not self.grid[new_row][new_col].is_blocked and avoid_condition:
                        fringe.append((new_row, new_col))
                        visited.add((new_row, new_col))
                        parent[(new_row, new_col)] = (row, col)

        return None # If path not found
   
    def move_bot_for_1_crew(self, crew_beep):
        # Initialize variables to store the maximum crew belief and the corresponding target cell
        max_crew_belief = 0.0
        target_cells = []
        avoid_alien = True
        best_neighbor = None
        min_alien_neighbor = float('inf')
        max_crew_neighbor = float('-inf')
        target_cell = None
        neighbors = self.open_neighbors.get((self.bot_location[0], self.bot_location[1]))

        crew_probs = np.array([[cell.prob_crew for cell in row] for row in self.grid])

        # Find the maximum probability value and its indices
        max_prob = crew_probs.max()
        max_indices = np.argwhere(crew_probs == max_prob)

        # Extract the coordinates of cells with the highest probability
        target_cells = [(i, j) for i, j in max_indices]
         # Prioritize moving towards high-probability crew member cells
        for neighbor in neighbors:
            i, j = neighbor
            crew_belief = self.grid[i][j].prob_crew
            alien_belief = self.grid[i][j].prob_alien
            if any(crew_beep):
                if crew_belief > max_crew_neighbor:
                    max_crew_neighbor = crew_belief
                    best_neighbor = neighbor
            else:
                if alien_belief < min_alien_neighbor:
                    min_alien_neighbor = alien_belief
                    best_neighbor = neighbor
        # Choose one of the target cells randomly if there are multiple cells with max crew belief
        if target_cells:
            target_cell = random.choice(target_cells)

        # print(f"bot move to {target_cell} with probability {max_crew_belief}")
        path = self.find_path(self.bot_location, target_cell, avoid_alien)
        if path is not None and len(path)>0:
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
            # Update the bot's location
            self.bot_location = path[0]
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
        else:
            avoid_alien = False
            path = self.find_path(self.bot_location, target_cell, avoid_alien)
            if path is not None and len(path)>0:
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                # Update the bot's location
                self.bot_location = path[0]
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
            else:
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                 # Update the bot's location
                self.bot_location = best_neighbor
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True

    # Movement decision
    def move_bot(self, crew_beep):
        if(self.crew_found != None):
            self.move_bot_for_1_crew(crew_beep)
            return
        # Determine cell most likely to contain crew member
        neighbors = self.open_neighbors.get((self.bot_location[0], self.bot_location[1]))
        avoid_alien = True
        max_crew_belief = 0.0
        best_neighbor = None
        min_alien_neighbor = float('inf')
        max_crew_neighbor = float('-inf')
        target_cell = None
        
        max_crew_belief_cells = max(self.crew_beliefs, key = self.crew_beliefs.get)
        # print('max prob cell', max_crew_belief_cells)
        max_crew_belief = self.crew_beliefs[max_crew_belief_cells]
        
        for neighbor in neighbors:
            i, j = neighbor
            crew_belief = self.grid[i][j].prob_crew
            alien_belief = self.grid[i][j].prob_alien
            if any(crew_beep):
                if crew_belief > max_crew_neighbor:
                    max_crew_neighbor = crew_belief
                    best_neighbor = neighbor
            else:
                if alien_belief < min_alien_neighbor:
                    min_alien_neighbor = alien_belief
                    best_neighbor = neighbor
        
        crew1_cell = max_crew_belief_cells[0]
        crew2_cell = max_crew_belief_cells[1]

        crew1_distance = self.get_distance(crew1_cell)
        crew2_distance = self.get_distance(crew2_cell)

        target_cell = crew1_cell if crew1_distance < crew2_distance and self.bot_location != crew1_cell else (crew2_cell if self.bot_location != crew2_cell else None)
        path = self.find_path(self.bot_location, target_cell, avoid_alien)

        if path is not None and len(path)>0:
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
            self.bot_location = path[0]
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
        else:
            avoid_alien = False
            path = self.find_path(self.bot_location, target_cell, avoid_alien)
            if path is not None and len(path)>0:
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                # Update the bot's location
                self.bot_location = path[0]
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
            else:
                # First attempt failed, try the other crew member's cell as the target
                other_target_cell = crew2_cell if target_cell == crew1_cell else crew1_cell
                path = self.find_path(self.bot_location, other_target_cell)
                if path is not None and len(path) > 0:
                    self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                    self.bot_location = path[0]
                    self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
                else:
                    self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                    # Update the bot's location
                    self.bot_location = best_neighbor
                    self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True            


    # Handling alien movement
    def handle_alien_movement(self):
        new_alien_positions = []
        for i in range(len(self.all_alien_locations)):
            x, y = self.all_alien_locations[i]
            directions = [(0,1),(0,-1),(-1,0),(1,0)]
            random.shuffle(directions)
            possible_moves = [
            (nx, ny) 
            for dx, dy in directions
            if 0 <= (nx := x + dx) < self.size and 0 <= (ny := y + dy) < self.size and self.grid[nx][ny].is_blocked == False and self.grid[nx][ny].is_Alien == False
            ]
            if(possible_moves):
                new_position = random.choice(possible_moves)
                new_alien_positions.append(new_position)
                ax, ay = new_position
                self.grid[ax][ay].is_Alien = True
                self.grid[x][y].is_Alien = False
        
        self.all_alien_locations = new_alien_positions
        # Update knowledge of alien's possible locations
        # Adjust movement strategy to avoid alien
        
    # main function
    def bot_function(self):
        steps = 1000
        self.Initialize_crew_beliefs()
        self.Initialize_alien_beliefs()
        num_of_crew_reached = 0
        for step in range(steps):
            # print("self.crew_locations self.bot_location self.alien_locations", self.crew_locations, self.bot_location, self.all_alien_locations)
            # Check if the bot reaches the crew
            crew_beep = []
            for crew_location in self.crew_locations:
                crew_beep.append(self.check_beep_for_crew(crew_location))
            Alien_detect = self.check_for_alien_beep()
            self.update_knowledge_crew(crew_beep)
            self.update_knowledge_alien(Alien_detect)
            self.move_bot(crew_beep)

            if self.bot_location in self.crew_locations:
                num_of_crew_reached += 1
                self.crew_found = self.bot_location
                if num_of_crew_reached == 2:
                    # print("Congratulations! Bot reached the crew in steps:", step)
                    return True, step, num_of_crew_reached
                else:
                    # print("Congratz, CREW1 Reached")
                    self.grid[self.bot_location[0]][self.bot_location[1]].is_crew =  False
                    self.crew_locations.remove(self.bot_location)
                    self.initialize_prob()
            
            # Check if the bot is caught by an alien
            if self.bot_location in self.all_alien_locations:
                # print("Game over! Bot caught by an alien in steps:", step)
                return False, step, num_of_crew_reached
            
            Alien_detect = self.check_for_alien_beep()
            self.update_knowledge_alien(Alien_detect)
            self.handle_alien_movement()
            # visualize_grid(self.grid)
            # Check if the bot reaches the crew
            if self.bot_location in self.all_alien_locations:
                # print("Game over! Bot caught by an alien in steps:", step)
                return False, step, num_of_crew_reached
            
        # print("Game over! Bot couldn't reach the crew in steps:", step)
        return False, step, num_of_crew_reached

In [ ]:
class Bot5:
    def __init__(self, D, k, alpha, ship):
        self.size = D
        self.ship = ship
        self.k = k
        self.grid = self.ship.grid
        self.bot_location = self.ship.bot_location
        self.all_alien_locations = self.ship.all_alien_locations
        self.crew_locations = self.ship.crew_locations
        self.alpha = alpha
        self.crew_beliefs = {}
        self.crew_found = None
        self.pos_with_high_alien_beliefs = []
        self.open_cells = self.ship.open_cells
        self.open_neighbors = self.ship.open_neighbors
        self.alien_threshold = 0.01
        self.prev_cells = deque(maxlen=15)
        self.prev_beeps = deque(maxlen=15)

    def Initialize_crew_beliefs(self):
        num_open_cells = len(self.open_cells)
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked and not self.grid[i][j].is_bot:
                    for m in range(self.size):
                        for n in range(self.size):
                            if not self.grid[m][n].is_blocked and not self.grid[i][j].is_bot and (i!=m and j!=n):
                                key = tuple(sorted(((i, j), (m, n))))  # Sort the tuple to ensure consistent order key = ((i, j), (m, n))
                                self.crew_beliefs[key] = 1/(num_open_cells-1)*1/(num_open_cells-1)

    def Initialize_alien_beliefs(self):
        k = 0
        # check for number of open cells in detection square 
        for i in range(self.size):
            for j in range(self.size):
                if self.in_detection_square((i,j)) and not self.grid[i][j].is_blocked:
                    k +=1
        n = len(self.open_cells)-k        
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked and not self.in_detection_square((i,j)):
                    self.grid[i][j].prob_alien = 1/(n)
    
    def check_beep_for_crew(self, crew_position):
        # Calculate Manhattan distance between bot and crew member
        distance = abs(self.bot_location[0] - crew_position[0]) + abs(self.bot_location[1] - crew_position[1])
        # Calculate probability of hearing the beep
        beep_prob = np.exp(-self.alpha * (distance - 1))
        rand_num = random.random()
        if rand_num <= beep_prob:
            return True  # Bot hears the beep
        else:
            return False  # Bot does not hear the beep

    def check_for_alien_beep(self):
        for i in range(-self.k, self.k+1):
            for j in range(-self.k, self.k+1):
                x = self.bot_location[0]+i
                y = self.bot_location[1]+j
                if 0<=x<self.size and 0<=y<self.size and self.grid[x][y].is_Alien:
                    return True
        return False 

    def get_distance(self, x):
        return  abs(self.bot_location[0] - x[0]) + abs(self.bot_location[1] - x[1])
    
    
    # Update bot's knowledge at each timestep
    def update_knowledge_crew(self, crew_beep):
        probability_sum = 0
        if self.crew_found == None:
            # if both crew are present we will keep track of pairwise probabilities
            for key in self.crew_beliefs:
                crew_i = key[0]
                crew_j = key[1]

                if self.bot_location in key:
                    self.crew_beliefs[(crew_i, crew_j)] = 0.0 #set belief to 0 for current bot location as we know crew member is not there  
                    continue  # Skip this iteration if bot location is part of key

                distance_to_crew_i = self.get_distance(crew_i)
                distance_to_crew_j = self.get_distance(crew_j)
                crew_i_prob = np.exp(-self.alpha * (distance_to_crew_i - 1))
                crew_j_prob = np.exp(-self.alpha * (distance_to_crew_j - 1))
                
                if any(crew_beep):
                    likelihood = crew_i_prob + crew_j_prob - crew_i_prob * crew_j_prob
                else:
                    likelihood = (1-crew_i_prob) * (1-crew_j_prob)
                            
                # Calculate prior probability P(crew_i and crew_j)
                prior_prob = self.crew_beliefs.get((crew_i, crew_j), 0.0)

                new_prob = prior_prob * likelihood
                self.crew_beliefs[(crew_i, crew_j)] = new_prob

                probability_sum += self.crew_beliefs[(crew_i, crew_j)]
            
            # Normalizing 
            normalized_beliefs = {}
            for key, value in self.crew_beliefs.items():
                normalized_beliefs[key] = value / probability_sum
    
            self.crew_beliefs = normalized_beliefs
        
        else:
        # go through each cell and update the probability of crew being present based on the beeps recieved
            for i in range(self.size):
                for j in range(self.size):
                    if self.grid[i][j].is_bot:
                        self.grid[i][j].prob_crew = 0.0
                    elif not self.grid[i][j].is_blocked:
                        distance = self.get_distance((i,j))
                        prob = np.exp(-self.alpha * (distance - 1))
                        # update belief of crew if beef recieved
                        if any(self.prev_beeps):
                            self.grid[i][j].prob_crew *= prob
                        else:
                            self.grid[i][j].prob_crew *= (1 - prob)
                    probability_sum +=self.grid[i][j].prob_crew
            
            # Normalizing 
            if probability_sum > 0:
                for i in range(self.size):
                    for j in range(self.size):
                        if not self.grid[i][j].is_blocked:
                            self.grid[i][j].prob_crew /= probability_sum
            else:
                self.initialize_prob()

            # Append current cell position and beep occurrence to deque
        self.prev_cells.append(self.bot_location)
        self.prev_beeps.append(any(crew_beep))

    def initialize_prob(self):
        n = len(self.open_cells)
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked and not self.grid[i][j].is_bot:
                    self.grid[i][j].prob_crew = 1/(n-1)  

    def negate_crew_prob(self):
        # Calculate reduced beep count
        reduced_beep_count = int(len(self.prev_beeps) * 0.5)

        # Make copies of relevant data structures for mock simulation
        prev_cells_copy = list(self.prev_cells)
        prev_beeps_copy = list(self.prev_beeps)[:reduced_beep_count]

        # Perform mock simulation for the last 15 time steps
        for _ in range(15):
            # Update bot's knowledge with reduced beep count
            probability_sum = 0
            for i in range(self.size):
                for j in range(self.size):
                    if self.grid[i][j].is_bot :
                        self.grid[i][j].prob_crew = 0.0
                    elif not self.grid[i][j].is_blocked:
                        distance = self.get_distance((i,j))
                        prob = np.exp(-self.alpha * (distance - 1))
                        if any(prev_beeps_copy):
                            self.grid[i][j].prob_crew *= prob
                        else:
                            self.grid[i][j].prob_crew *= (1 - prob)
                        probability_sum += self.grid[i][j].prob_crew

            # Normalize probabilities
            if probability_sum > 0:
                for i in range(self.size):
                    for j in range(self.size):
                        if not self.grid[i][j].is_blocked:
                            self.grid[i][j].prob_crew /= probability_sum


            # Move bot randomly
            self.move_bot_randomly()

            # Update previous cells and beeps
            self.prev_cells.append(self.bot_location)
            self.prev_beeps.append(any(prev_beeps_copy))
            prev_cells_copy.append(self.bot_location)
            prev_beeps_copy.append(any(prev_beeps_copy))
            prev_cells_copy = prev_cells_copy[-15:]
            prev_beeps_copy = prev_beeps_copy[-reduced_beep_count:]
        

        # Update crew probability distribution based on mock simulation
        for cell in prev_cells_copy:
            i, j = cell
            if not self.grid[i][j].is_blocked:
                distance = self.get_distance(cell)
                prob = np.exp(-self.alpha * (distance - 1))
                if any(prev_beeps_copy):
                    self.grid[i][j].prob_crew *= prob
                else:
                    self.grid[i][j].prob_crew *= (1 - prob)
        
        # Normalize crew probability distribution
        probability_sum = sum(cell.prob_crew for row in self.grid for cell in row if not cell.is_blocked)
        if probability_sum >0:
            for i in range(self.size):
                for j in range(self.size):
                    if not self.grid[i][j].is_blocked:
                        self.grid[i][j].prob_crew /= probability_sum


    def in_detection_square(self, x):
        return abs(x[0] - self.bot_location[0]) <= self.k and abs(x[1] - self.bot_location[1]) <= self.k

    def update_knowledge_alien(self, Alien_detect):
        alien_prob = np.zeros((self.size, self.size), dtype=float)
        total_prob_alien = 0
        # if we have detected the alien that means bot is inside the detection square and only probabilities of cells inside the detection square needs to be updated
        if Alien_detect:
            for i in range(self.size):
                for j in range(self.size):
                    if not self.grid[i][j].is_blocked and self.in_detection_square((i,j)) and not self.grid[i][j].is_bot:
                      # alien is inside the detection square that means it either came from outside or it was inside in previous timestamp. 
                        #in both cases we need to update the cells probability based on the probability of its neighbor as alien moved from its immediate neighbors that's why we heard the detection beep 
                        neighbors = self.open_neighbors.get((i,j))
                        for n in neighbors:
                          # for all neighbors we need to have their transitional probability as well that is with the probability they moved to that particular cell
                            valid_neighbors = self.open_neighbors.get((n[0],n[1]))
                            total_neighbors = len(valid_neighbors)
                            if total_neighbors > 0:
                                alien_prob[i][j] += self.grid[n[0]][n[1]].prob_alien/(total_neighbors+1)
                        total_prob_alien += alien_prob[i][j]

        else:
            # No alien beep detected, update all cells to have zero probability of alien presence
            for i in range(self.size):
                for j in range(self.size):
                    if not self.grid[i][j].is_blocked and not self.in_detection_square((i,j)):
                        neighbors = self.open_neighbors.get((i,j))
                        for n in neighbors:
                          # for all neighbors we need to have their transitional probability as well that is with the probability they moved to that particular cell
                            valid_neighbors = self.open_neighbors.get((n[0],n[1]))
                            total_neighbors = len(valid_neighbors)
                            if total_neighbors > 0:
                                alien_prob[i][j] += self.grid[n[0]][n[1]].prob_alien/(total_neighbors+1)
                        total_prob_alien += alien_prob[i][j]

        
       # Normalize probabilities if total_prob_alien is not zero
        if total_prob_alien != 0:
            for i in range(self.size):
                for j in range(self.size):
                    if not self.grid[i][j].is_blocked:
                        self.grid[i][j].prob_alien = alien_prob[i][j]/total_prob_alien
        
        else:
             self.Initialize_alien_beliefs()  # Reset beliefs if total probability is zero

        alien_probabilities = [(i, j) for i in range(self.size) for j in range(self.size)]
        sorted_alien_probabilities = sorted(alien_probabilities, key=lambda x: self.grid[x[0]][x[1]].prob_alien, reverse=True)
        self.pos_with_high_alien_beliefs = sorted_alien_probabilities[:5]


    def find_path(self, src, dest, avoid_alien = True):
        # print("started path planning from", src, "to " , dest)
        visited = set() # Dictionary to store the visited nodes
        parent = {}  # Dictionary to store parent nodes
        # Keep track of the nodes
        fringe = deque([src])
        
        while fringe:
            (row, col) = fringe.popleft()
            # Process the current cell until it reaches the destination
            if (row, col) == dest:
                # Reconstruct the path from dest to src
                path = []
                while (row, col) != src:
                    path.append((row, col))
                    row, col = parent[(row, col)]
                path.reverse()
                return path  # Path found
            
            # Enqueue for unvisited neighbors
            for dr, dc in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                new_row, new_col = row + dr, col + dc
                # Check if the new cell is within the boundaries 0 and length of Ship
                # Check if the new cell is not in alien location and not in visited
                # Check if the new cell is not blocked 
                if 0 <= new_row < self.size and 0 <= new_col < self.size:
                    avoid_condition =  (self.grid[new_row][new_col].prob_alien == 0.0) or (not avoid_alien and (new_row, new_col) not in self.pos_with_high_alien_beliefs)
                    if (new_row, new_col) not in visited and not self.grid[new_row][new_col].is_blocked and avoid_condition:
                        fringe.append((new_row, new_col))
                        visited.add((new_row, new_col))
                        parent[(new_row, new_col)] = (row, col)

        return None # If path not found
   
    def move_bot_for_1_crew(self, crew_beep):
        # Initialize variables to store the maximum crew belief and the corresponding target cell
        max_crew_belief = 0.0
        target_cells = []
        avoid_alien = True
        best_neighbor = None
        target_cell = None
        min_alien_neighbor = float('inf')
        max_crew_neighbor = float('-inf')
        neighbors = self.open_neighbors.get((self.bot_location[0], self.bot_location[1]))

        crew_probs = np.array([[cell.prob_crew for cell in row] for row in self.grid])

        # Find the maximum probability value and its indices
        max_prob = crew_probs.max()
        max_indices = np.argwhere(crew_probs == max_prob)

        # Extract the coordinates of cells with the highest probability
        target_cells = [(i, j) for i, j in max_indices]
         # Prioritize moving towards high-probability crew member cells
        for neighbor in neighbors:
            i, j = neighbor
            crew_belief = self.grid[i][j].prob_crew
            alien_belief = self.grid[i][j].prob_alien
            if any(crew_beep):
                if crew_belief > max_crew_neighbor:
                    max_crew_neighbor = crew_belief
                    best_neighbor = neighbor
            else:
                if alien_belief < min_alien_neighbor:
                    min_alien_neighbor = alien_belief
                    best_neighbor = neighbor
        # Choose one of the target cells randomly if there are multiple cells with max crew belief
        if target_cells:
            target_cell = random.choice(target_cells)

        # print(f"bot move to {target_cell} with probability {max_crew_belief}")
        path = self.find_path(self.bot_location, target_cell, avoid_alien)
        if path is not None and len(path)>0:
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
            # Update the bot's location
            self.bot_location = path[0]
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
        else:
            avoid_alien = False
            path = self.find_path(self.bot_location, target_cell, avoid_alien)
            if path is not None and len(path)>0:
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                # Update the bot's location
                self.bot_location = path[0]
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
            else:
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                 # Update the bot's location
                self.bot_location = best_neighbor
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True

    def move_bot_randomly(self):
        # Define possible movements
        directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Right, Left, Down, Up

        # Choose a random movement
        movement = random.choice(directions)
        new_location = (self.bot_location[0] + movement[0], self.bot_location[1] + movement[1])
        print(self.bot_location)
        if 0 <= new_location[0] < self.size and 0 <= new_location[1] < self.size:
            # Update bot's location
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
            self.bot_location = new_location
            print(self.bot_location)
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True


    # Movement decision
    def move_bot(self, crew_beep):
        if(self.crew_found != None):
            self.move_bot_for_1_crew(crew_beep)
            return
        # Determine cell most likely to contain crew member
        neighbors = self.open_neighbors.get((self.bot_location[0], self.bot_location[1]))
        avoid_alien = True
        max_crew_belief = 0.0
        best_neighbor = None
        min_alien_neighbor = float('inf')
        max_crew_neighbor = float('-inf')
        target_cell = None
        
        max_crew_belief_cells = max(self.crew_beliefs, key = self.crew_beliefs.get)
        # print('max prob cell', max_crew_belief_cells)
        max_crew_belief = self.crew_beliefs[max_crew_belief_cells]
        
        for neighbor in neighbors:
            i, j = neighbor
            crew_belief = self.grid[i][j].prob_crew
            alien_belief = self.grid[i][j].prob_alien
            if any(crew_beep):
                if crew_belief > max_crew_neighbor:
                    max_crew_neighbor = crew_belief
                    best_neighbor = neighbor
            else:
                if alien_belief < min_alien_neighbor:
                    min_alien_neighbor = alien_belief
                    best_neighbor = neighbor
        
        crew1_cell = max_crew_belief_cells[0]
        crew2_cell = max_crew_belief_cells[1]

        crew1_distance = self.get_distance(crew1_cell)
        crew2_distance = self.get_distance(crew2_cell)

        target_cell = crew1_cell if crew1_distance < crew2_distance and self.bot_location != crew1_cell else (crew2_cell if self.bot_location != crew2_cell else None)
        path = self.find_path(self.bot_location, target_cell, avoid_alien)

        if path is not None and len(path)>0:
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
            self.bot_location = path[0]
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
        else:
            avoid_alien = False
            path = self.find_path(self.bot_location, target_cell, avoid_alien)
            if path is not None and len(path)>0:
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                # Update the bot's location
                self.bot_location = path[0]
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
            else:
                # First attempt failed, try the other crew member's cell as the target
                other_target_cell = crew2_cell if target_cell == crew1_cell else crew1_cell
                path = self.find_path(self.bot_location, other_target_cell)
                if path is not None and len(path) > 0:
                    self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                    self.bot_location = path[0]
                    self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
                else:
                    self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                    # Update the bot's location
                    self.bot_location = best_neighbor
                    self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True            


    # Handling alien movement
    def handle_alien_movement(self):
        new_alien_positions = []
        for i in range(len(self.all_alien_locations)):
            x, y = self.all_alien_locations[i]
            directions = [(0,1),(0,-1),(-1,0),(1,0)]
            random.shuffle(directions)
            possible_moves = [
            (nx, ny) 
            for dx, dy in directions
            if 0 <= (nx := x + dx) < self.size and 0 <= (ny := y + dy) < self.size and self.grid[nx][ny].is_blocked == False and self.grid[nx][ny].is_Alien == False
            ]
            if(possible_moves):
                new_position = random.choice(possible_moves)
                new_alien_positions.append(new_position)
                ax, ay = new_position
                self.grid[ax][ay].is_Alien = True
                self.grid[x][y].is_Alien = False
        
        self.all_alien_locations = new_alien_positions
        # Update knowledge of alien's possible locations
        # Adjust movement strategy to avoid alien
        
    # main function
    def bot_function(self):
        steps = 1000
        self.Initialize_crew_beliefs()
        self.Initialize_alien_beliefs()
        num_of_crew_reached = 0
        for step in range(steps):
            # print("self.crew_locations self.bot_location self.alien_locations", self.crew_locations, self.bot_location, self.all_alien_locations)
            # Check if the bot reaches the crew
            crew_beep = []
            for crew_location in self.crew_locations:
                crew_beep.append(self.check_beep_for_crew(crew_location))
            Alien_detect = self.check_for_alien_beep()
            self.update_knowledge_crew(crew_beep)
            self.update_knowledge_alien(Alien_detect)
            self.move_bot(crew_beep)

            if self.bot_location in self.crew_locations:
                num_of_crew_reached += 1
                self.crew_found = self.bot_location
                if num_of_crew_reached == 2:
                    # print("Congratulations! Bot reached the crew in steps:", step)
                    return True, step, num_of_crew_reached
                else:
                    # print("Congratz, CREW1 Reached")
                    self.grid[self.bot_location[0]][self.bot_location[1]].is_crew =  False
                    self.crew_locations.remove(self.bot_location)
                    crew_prob_matrix = np.zeros((self.size, self.size))

                    # Update each cell's probability to be the sum of probabilities of all combinations of crew members
                    for i in range(self.size):
                        for j in range(self.size):
                            total_prob = 0
                            for (x, y), prob in self.crew_beliefs.items():
                                if (x == i and y == j) or (x == j and y == i):
                                    total_prob += prob
                            crew_prob_matrix[i][j] = total_prob
                            
                    for i in range(self.size):
                        for j in range(self.size):
                            self.grid[i][j].prob_crew = crew_prob_matrix[i][j]

                    self.negate_crew_prob()
            
            # Check if the bot is caught by an alien
            if self.bot_location in self.all_alien_locations:
                # print("Game over! Bot caught by an alien in steps:", step)
                return False, step, num_of_crew_reached
            
            Alien_detect = self.check_for_alien_beep()
            self.update_knowledge_alien(Alien_detect)
            self.handle_alien_movement()
            # visualize_grid(self.grid)
            # Check if the bot reaches the crew
            if self.bot_location in self.all_alien_locations:
                # print("Game over! Bot caught by an alien in steps:", step)
                return False, step, num_of_crew_reached
            
        # print("Game over! Bot couldn't reach the crew in steps:", step)
        return False, step, num_of_crew_reached

In [ ]:
class Bot6:
    def __init__(self, D, k, alpha, ship):
        self.size = D
        self.ship = ship
        self.k = k
        self.grid = self.ship.grid
        self.bot_location = self.ship.bot_location
        self.all_alien_locations = self.ship.all_alien_locations
        self.crew_locations = self.ship.crew_locations
        self.alpha = alpha
        self.crew_beliefs = {}
        self.crew_found = None
        self.pos_with_high_alien_beliefs = []
        self.open_cells = self.ship.open_cells
        self.open_neighbors = self.ship.open_neighbors
        self.alien_threshold = 0.01
        self.prev_cells = deque(maxlen=15)
        self.prev_beeps = deque(maxlen=15)

    def Initialize_crew_beliefs(self):
        num_open_cells = len(self.open_cells)
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked and not self.grid[i][j].is_bot:
                    for m in range(self.size):
                        for n in range(self.size):
                            if not self.grid[m][n].is_blocked and not self.grid[i][j].is_bot and (i!=m and j!=n):
                                key = tuple(sorted(((i, j), (m, n))))  # Sort the tuple to ensure consistent order key = ((i, j), (m, n))
                                self.crew_beliefs[key] = 1/(num_open_cells-1)*1/(num_open_cells-1)

    def Initialize_alien_beliefs(self):
        k = 0
        # check for number of open cells in detection square 
        for i in range(self.size):
            for j in range(self.size):
                if self.in_detection_square((i,j)) and not self.grid[i][j].is_blocked:
                    k +=1
        n = len(self.open_cells)-k        
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked and not self.in_detection_square((i,j)):
                    self.grid[i][j].prob_alien = 1/(n)
    
    def check_beep_for_crew(self, crew_position):
        # Calculate Manhattan distance between bot and crew member
        distance = abs(self.bot_location[0] - crew_position[0]) + abs(self.bot_location[1] - crew_position[1])
        # Calculate probability of hearing the beep
        beep_prob = np.exp(-self.alpha * (distance - 1))
        rand_num = random.random()
        if rand_num <= beep_prob:
            return True  # Bot hears the beep
        else:
            return False  # Bot does not hear the beep

    def check_for_alien_beep(self):
        for i in range(-self.k, self.k+1):
            for j in range(-self.k, self.k+1):
                x = self.bot_location[0]+i
                y = self.bot_location[1]+j
                if 0<=x<self.size and 0<=y<self.size and self.grid[x][y].is_Alien:
                    return True
        return False 

    def get_distance(self, x):
        return  abs(self.bot_location[0] - x[0]) + abs(self.bot_location[1] - x[1])
    
    
    # Update bot's knowledge at each timestep
    def update_knowledge_crew(self, crew_beep):
        probability_sum = 0
        if self.crew_found == None:
            for key in self.crew_beliefs:
                crew_i = key[0]
                crew_j = key[1]

                if self.bot_location in key:
                    self.crew_beliefs[(crew_i, crew_j)] = 0.0 #set belief to 0 for current bot location as we know crew member is not there  
                    continue  # Skip this iteration if bot location is part of key

                distance_to_crew_i = self.get_distance(crew_i)
                distance_to_crew_j = self.get_distance(crew_j)
                 # P(beep in bot's cell | crew in cell i)
                crew_i_prob = np.exp(-self.alpha * (distance_to_crew_i - 1))
                # P(beep in bot's cell | crew in cell j)
                crew_j_prob = np.exp(-self.alpha * (distance_to_crew_j - 1))
                
                if any(crew_beep):
                    likelihood = crew_i_prob + crew_j_prob - crew_i_prob * crew_j_prob
                else:
                    likelihood = (1-crew_i_prob) * (1-crew_j_prob)
                            
                # Calculate prior probability P(crew_i and crew_j)
                prior_prob = self.crew_beliefs.get((crew_i, crew_j), 0.0)

                new_prob = prior_prob * likelihood
                self.crew_beliefs[(crew_i, crew_j)] = new_prob

                probability_sum += self.crew_beliefs[(crew_i, crew_j)]
            
            # Normalizing 
            normalized_beliefs = {}
            for key, value in self.crew_beliefs.items():
                normalized_beliefs[key] = value / probability_sum
    
            self.crew_beliefs = normalized_beliefs
        
        else:
        # go through each cell and update the probability of crew being present based on the beeps recieved
            for i in range(self.size):
                for j in range(self.size):
                    if self.grid[i][j].is_bot:
                        self.grid[i][j].prob_crew = 0.0
                    elif not self.grid[i][j].is_blocked:
                        distance = self.get_distance((i,j))
                        prob = np.exp(-self.alpha * (distance - 1))
                        # update belief of crew if beef recieved
                        if any(crew_beep):
                            self.grid[i][j].prob_crew *= prob
                        else:
                            self.grid[i][j].prob_crew *= (1 - prob)
                    probability_sum +=self.grid[i][j].prob_crew
            
            # Normalizing 
            if probability_sum > 0:
                for i in range(self.size):
                    for j in range(self.size):
                        if not self.grid[i][j].is_blocked:
                            self.grid[i][j].prob_crew /= probability_sum
            
            else:
                self.initialize_prob()

                # Append current cell position and beep occurrence to deque
        self.prev_cells.append(self.bot_location)
        self.prev_beeps.append(any(crew_beep))

    def initialize_prob(self):
        n = len(self.open_cells)
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked and not self.grid[i][j].is_bot:
                    self.grid[i][j].prob_crew = 1/(n-1)

    def negate_crew_prob(self):
        # Calculate reduced beep count
        reduced_beep_count = int(len(self.prev_beeps) * 0.5)

        # Make copies of relevant data structures for mock simulation
        prev_cells_copy = list(self.prev_cells)
        prev_beeps_copy = list(self.prev_beeps)[:reduced_beep_count]

        # Perform mock simulation for the last 15 time steps
        for _ in range(15):
            # Update bot's knowledge with reduced beep count
            probability_sum = 0
            for i in range(self.size):
                for j in range(self.size):
                    if self.grid[i][j].is_bot:
                        self.grid[i][j].prob_crew = 0.0
                    elif not self.grid[i][j].is_blocked:
                        distance = self.get_distance((i,j))
                        prob = np.exp(-self.alpha * (distance - 1))
                        if any(prev_beeps_copy):
                            self.grid[i][j].prob_crew *= prob
                        else:
                            self.grid[i][j].prob_crew *= (1 - prob)
                        probability_sum += self.grid[i][j].prob_crew

            # Normalize probabilities
            if probability_sum > 0:
                for i in range(self.size):
                    for j in range(self.size):
                        if not self.grid[i][j].is_blocked:
                            self.grid[i][j].prob_crew /= probability_sum

            # Move bot randomly
            self.move_bot_randomly()

            # Update previous cells and beeps
            self.prev_cells.append(self.bot_location)
            self.prev_beeps.append(any(prev_beeps_copy))
            prev_cells_copy.append(self.bot_location)
            prev_beeps_copy.append(any(prev_beeps_copy))
            prev_cells_copy = prev_cells_copy[-15:]
            prev_beeps_copy = prev_beeps_copy[-reduced_beep_count:]

        # Update crew probability distribution based on mock simulation
        for cell in prev_cells_copy:
            i, j = cell
            if not self.grid[i][j].is_blocked:
                distance = self.get_distance(cell)
                prob = np.exp(-self.alpha * (distance - 1))
                if any(prev_beeps_copy):
                    self.grid[i][j].prob_crew *= prob
                else:
                    self.grid[i][j].prob_crew *= (1 - prob)

        # Normalize crew probability distribution
        probability_sum = sum(cell.prob_crew for row in self.grid for cell in row if not cell.is_blocked)
        if probability_sum > 0:
            for i in range(self.size):
                for j in range(self.size):
                    if not self.grid[i][j].is_blocked:
                        self.grid[i][j].prob_crew /= probability_sum

    def move_bot_randomly(self):
    # Define possible movements
        directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Right, Left, Down, Up

        # Choose a random movement
        movement = random.choice(directions)
        new_location = (self.bot_location[0] + movement[0], self.bot_location[1] + movement[1])

        if 0 <= new_location[0] < self.size and 0 <= new_location[1] < self.size:
            # Update bot's location
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
            self.bot_location = new_location
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True

         
    def in_detection_square(self, x):
        return abs(x[0] - self.bot_location[0]) <= self.k and abs(x[1] - self.bot_location[1]) <= self.k

    def update_knowledge_alien(self, Alien_detect):
        alien_prob = np.zeros((self.size, self.size), dtype=float)
        total_prob_alien = 0
        # if we have detected the alien that means bot is inside the detection square and only probabilities of cells inside the detection square needs to be updated
        if any(Alien_detect):
            for i in range(self.size):
                for j in range(self.size):
                    if not self.grid[i][j].is_blocked and self.in_detection_square((i,j)) and not self.grid[i][j].is_bot:
                      # alien is inside the detection square that means it either came from outside or it was inside in previous timestamp. 
                        #in both cases we need to update the cells probability based on the probability of its neighbor as alien moved from its immediate neighbors that's why we heard the detection beep 
                        neighbors = self.open_neighbors.get((i,j))
                        for n in neighbors:
                          # for all neighbors we need to have their transitional probability as well that is with the probability they moved to that particular cell
                            valid_neighbors = self.open_neighbors.get((n[0],n[1]))
                            total_neighbors = len(valid_neighbors)
                            if total_neighbors > 0:
                                alien_prob[i][j] += self.grid[n[0]][n[1]].prob_alien/(total_neighbors+1)
                        total_prob_alien += alien_prob[i][j]

        else:
            # No alien beep detected, update all cells to have zero probability of alien presence
            for i in range(self.size):
                for j in range(self.size):
                    if not self.grid[i][j].is_blocked and not self.in_detection_square((i,j)):
                        neighbors = self.open_neighbors.get((i,j))
                        for n in neighbors:
                          # for all neighbors we need to have their transitional probability as well that is with the probability they moved to that particular cell
                            valid_neighbors = self.open_neighbors.get((n[0],n[1]))
                            total_neighbors = len(valid_neighbors)
                            if total_neighbors > 0:
                                alien_prob[i][j] += self.grid[n[0]][n[1]].prob_alien/(total_neighbors+1)
                        total_prob_alien += alien_prob[i][j]

        
        # Normalize probabilities if total_prob_alien is not zero
        if total_prob_alien != 0:
            for i in range(self.size):
                for j in range(self.size):
                    if not self.grid[i][j].is_blocked:
                        self.grid[i][j].prob_alien = alien_prob[i][j]/total_prob_alien
        
        else:
             self.Initialize_alien_beliefs()  # Reset beliefs if total probability is zero

        alien_probabilities = [(i, j) for i in range(self.size) for j in range(self.size)]
        sorted_alien_probabilities = sorted(alien_probabilities, key=lambda x: self.grid[x[0]][x[1]].prob_alien, reverse=True)
        self.pos_with_high_alien_beliefs = sorted_alien_probabilities[:5]

    
    def find_path(self, src, dest, avoid_alien = True):
        # print("started path planning from", src, "to " , dest)
        visited = set() # Dictionary to store the visited nodes
        parent = {}  # Dictionary to store parent nodes
        # Keep track of the nodes
        fringe = deque([src])
        
        while fringe:
            (row, col) = fringe.popleft()
            # Process the current cell until it reaches the destination
            if (row, col) == dest:
                # Reconstruct the path from dest to src
                path = []
                while (row, col) != src:
                    path.append((row, col))
                    row, col = parent[(row, col)]
                path.reverse()
                return path  # Path found
            
            # Enqueue for unvisited neighbors
            for dr, dc in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                new_row, new_col = row + dr, col + dc
                # Check if the new cell is within the boundaries 0 and length of Ship
                # Check if the new cell is not in alien location and not in visited
                # Check if the new cell is not blocked 
                if 0 <= new_row < self.size and 0 <= new_col < self.size:
                    avoid_condition =  (self.grid[new_row][new_col].prob_alien < self.alien_threshold) or (not avoid_alien and (new_row, new_col) not in self.pos_with_high_alien_beliefs)
                    if (new_row, new_col) not in visited and not self.grid[new_row][new_col].is_blocked and avoid_condition:
                        fringe.append((new_row, new_col))
                        visited.add((new_row, new_col))
                        parent[(new_row, new_col)] = (row, col)

        return None # If path not found
   
    def move_bot_for_1_crew(self, crew_beep):
        # Initialize variables to store the maximum crew belief and the corresponding target cell
        max_crew_belief = 0.0
        target_cells = []
        avoid_alien = True
        best_neighbor = None
        min_alien_neighbor = float('inf')
        max_crew_neighbor = float('-inf')
        
        neighbors = self.open_neighbors.get((self.bot_location[0], self.bot_location[1]))

         # Prioritize moving towards high-probability crew member cells
        crew_probs = np.array([[cell.prob_crew for cell in row] for row in self.grid])

        # Find the maximum probability value and its indices
        max_prob = crew_probs.max()
        max_indices = np.argwhere(crew_probs == max_prob)

        # Extract the coordinates of cells with the highest probability
        target_cells = [(i, j) for i, j in max_indices]
         # Prioritize moving towards high-probability crew member cells
        for neighbor in neighbors:
            i, j = neighbor
            crew_belief = self.grid[i][j].prob_crew
            alien_belief = self.grid[i][j].prob_alien
            if any(crew_beep):
                if crew_belief > max_crew_neighbor:
                    max_crew_neighbor = crew_belief
                    best_neighbor = neighbor
            else:
                if alien_belief < min_alien_neighbor:
                    min_alien_neighbor = alien_belief
                    best_neighbor = neighbor
                    
        # Choose one of the target cells randomly if there are multiple cells with max crew belief
        if target_cells:
            target_cell = random.choice(target_cells)

        # print(f"bot move to {target_cell} with probability {max_crew_belief}")
        path = self.find_path(self.bot_location, target_cell, avoid_alien)
        if path is not None and len(path)>0:
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
            # Update the bot's location
            self.bot_location = path[0]
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
        else:
            avoid_alien = False
            path = self.find_path(self.bot_location, target_cell, avoid_alien)
            if path is not None and len(path)>0:
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                # Update the bot's location
                self.bot_location = path[0]
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
            else:
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                 # Update the bot's location
                self.bot_location = best_neighbor
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True

    # Movement decision
    def move_bot(self, crew_beep):
        if(self.crew_found != None):
            self.move_bot_for_1_crew(crew_beep)
            return
        # Determine cell most likely to contain crew member
        neighbors = self.open_neighbors.get((self.bot_location[0], self.bot_location[1]))
        avoid_alien = True
        max_crew_belief = 0.0
        best_neighbor = None
        min_alien_neighbor = float('inf')
        max_crew_neighbor = float('-inf')
        target_cell = None
        
        max_crew_belief_cells = max(self.crew_beliefs, key = self.crew_beliefs.get)
        # print('max prob cell', max_crew_belief_cells)
        max_crew_belief = self.crew_beliefs[max_crew_belief_cells]
        
        for neighbor in neighbors:
            i, j = neighbor
            crew_belief = self.grid[i][j].prob_crew
            alien_belief = self.grid[i][j].prob_alien
            if any(crew_beep):
                if crew_belief > max_crew_neighbor:
                    max_crew_neighbor = crew_belief
                    best_neighbor = neighbor
            else:
                if alien_belief < min_alien_neighbor:
                    min_alien_neighbor = alien_belief
                    best_neighbor = neighbor
        
        crew1_cell = max_crew_belief_cells[0]
        crew2_cell = max_crew_belief_cells[1]

        crew1_distance = self.get_distance(crew1_cell)
        crew2_distance = self.get_distance(crew2_cell)

        target_cell = crew1_cell if crew1_distance < crew2_distance and self.bot_location != crew1_cell else (crew2_cell if self.bot_location != crew2_cell else None)
        path = self.find_path(self.bot_location, target_cell, avoid_alien)

        if path is not None and len(path)>0:
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
            self.bot_location = path[0]
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
        else:
            avoid_alien = False
            path = self.find_path(self.bot_location, target_cell, avoid_alien)
            if path is not None and len(path)>0:
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                # Update the bot's location
                self.bot_location = path[0]
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
            else:
                # First attempt failed, try the other crew member's cell as the target
                other_target_cell = crew2_cell if target_cell == crew1_cell else crew1_cell
                path = self.find_path(self.bot_location, other_target_cell)
                if path is not None and len(path) > 0:
                    self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                    self.bot_location = path[0]
                    self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
                else:
                    self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                    # Update the bot's location
                    self.bot_location = best_neighbor
                    self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True     

    def initialize_prob(self):
        n = len(self.open_cells)
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked and not self.grid[i][j].is_bot:
                    self.grid[i][j].prob_crew = 1/(n-1)       

    # Handling alien movement
    def handle_alien_movement(self):
        new_alien_positions = []
        for i in range(len(self.all_alien_locations)):
            x, y = self.all_alien_locations[i]
            directions = [(0,1),(0,-1),(-1,0),(1,0)]
            random.shuffle(directions)
            possible_moves = [
            (nx, ny) 
            for dx, dy in directions
            if 0 <= (nx := x + dx) < self.size and 0 <= (ny := y + dy) < self.size and self.grid[nx][ny].is_blocked == False and self.grid[nx][ny].is_Alien == False
            ]
            if(possible_moves):
                new_position = random.choice(possible_moves)
                new_alien_positions.append(new_position)
                ax, ay = new_position
                self.grid[ax][ay].is_Alien = True
                self.grid[x][y].is_Alien = False
        
        self.all_alien_locations = new_alien_positions
        
    # main function
    def bot_function(self):
        steps = 1000
        self.Initialize_crew_beliefs()
        self.Initialize_alien_beliefs()
        num_of_crew_reached = 0
        for step in range(steps):
            # print("self.crew_locations self.bot_location self.alien_locations", self.crew_locations, self.bot_location, self.all_alien_locations)
            # Check if the bot reaches the crew
            crew_beep = []
            Alien_detect = []
            # Alien_detect_1 = self.in_detection_square(self.all_alien_locations[0])
            # Alien_detect_2 = self.in_detection_square(self.all_alien_locations[1])
            for crew_location in self.crew_locations:
                crew_beep.append(self.check_beep_for_crew(crew_location))
            for alien in self.all_alien_locations:
                Alien_detect.append(self.in_detection_square(alien))
            # print(Alien_detect)
            self.update_knowledge_crew(crew_beep)
            self.update_knowledge_alien(Alien_detect)
            self.move_bot(crew_beep)

            if self.bot_location in self.crew_locations:
                num_of_crew_reached += 1
                self.crew_found = self.bot_location
                if num_of_crew_reached == 2:
                    # print("Congratulations! Bot reached the crew in steps:", step)
                    return True, step, num_of_crew_reached
                else:
                    # print("Congratz, CREW1 Reached")
                    self.grid[self.bot_location[0]][self.bot_location[1]].is_crew =  False
                    self.crew_locations.remove(self.bot_location)
                    crew_prob_matrix = np.zeros((self.size, self.size))

                    # Update each cell's probability to be the sum of probabilities of all combinations of crew members
                    for i in range(self.size):
                        for j in range(self.size):
                            total_prob = 0
                            for (x, y), prob in self.crew_beliefs.items():
                                if (x == i and y == j) or (x == j and y == i):
                                    total_prob += prob
                            crew_prob_matrix[i][j] = total_prob

                    for i in range(self.size):
                        for j in range(self.size):
                            self.grid[i][j].prob_crew = crew_prob_matrix[i][j]

                    self.negate_crew_prob()

            
            # Check if the bot is caught by an alien
            if self.bot_location in self.all_alien_locations:
                # print("Game over! Bot caught by an alien in steps:", step)
                return False, step, num_of_crew_reached
            
            for alien in self.all_alien_locations:
                Alien_detect.append(self.in_detection_square(alien))
            
            self.update_knowledge_alien(Alien_detect)
            self.handle_alien_movement()
            # visualize_grid(self.grid)
            # Check if the bot reaches the crew
            if self.bot_location in self.all_alien_locations:
                # print("Game over! Bot caught by an alien in steps:", step)
                return False, step, num_of_crew_reached
            
        # print("Game over! Bot couldn't reach the crew in steps:", step)
        return False, step, num_of_crew_reached

In [ ]:
class Bot7:
    def __init__(self, D, k, alpha, ship):
        self.size = D
        self.ship = ship
        self.k = k
        self.grid = self.ship.grid
        self.bot_location = self.ship.bot_location
        self.all_alien_locations = self.ship.all_alien_locations
        self.crew_locations = self.ship.crew_locations
        self.alpha = alpha
        self.crew_beliefs = {}
        self.crew_found = None
        self.alien_probabilities = {}  # Dictionary to store pairwise alien probabilities
        self.pos_with_high_alien_beliefs = []
        self.open_cells = self.ship.open_cells
        self.open_neighbors = self.ship.open_neighbors
        self.alien_threshold = 0.01
        self.prev_cells = deque(maxlen=15)
        self.prev_beeps = deque(maxlen=15)

    def Initialize_crew_beliefs(self):
        num_open_cells = len(self.open_cells)
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked and not self.grid[i][j].is_bot:
                    for m in range(self.size):
                        for n in range(self.size):
                            if not self.grid[m][n].is_blocked and not self.grid[i][j].is_bot and (i!=m and j!=n):
                                key = tuple(sorted(((i, j), (m, n))))  # Sort the tuple to ensure consistent order key = ((i, j), (m, n))
                                self.crew_beliefs[key] = 1/(num_open_cells-1)*1/(num_open_cells-1)

    def Initialize_alien_beliefs(self):
        k = 0
        # check for number of open cells in detection square 
        for i in range(self.size):
            for j in range(self.size):
                if self.in_detection_square((i,j)) and not self.grid[i][j].is_blocked:
                    k +=1
        num_open_cells = len(self.open_cells)-k        
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked:
                    for m in range(self.size):
                        for n in range(self.size):
                            if not self.grid[m][n].is_blocked and (i != m or j != n):
                                key = tuple(sorted(((i, j), (m, n))))  # Sort the tuple to ensure consistent order
                                self.alien_probabilities[key] = 1/(num_open_cells-1)*1/(num_open_cells-1)
    
    def check_beep_for_crew(self, crew_position):
        # Calculate Manhattan distance between bot and crew member
        distance = abs(self.bot_location[0] - crew_position[0]) + abs(self.bot_location[1] - crew_position[1])
        # Calculate probability of hearing the beep
        beep_prob = np.exp(-self.alpha * (distance - 1))
        rand_num = random.random()
        if rand_num <= beep_prob:
            return True  # Bot hears the beep
        else:
            return False  # Bot does not hear the beep

    def check_for_alien_beep(self):
        for i in range(-self.k, self.k+1):
            for j in range(-self.k, self.k+1):
                x = self.bot_location[0]+i
                y = self.bot_location[1]+j
                if 0<=x<self.size and 0<=y<self.size and self.grid[x][y].is_Alien:
                    return True
        return False 

    def get_distance(self, x):
        return  abs(self.bot_location[0] - x[0]) + abs(self.bot_location[1] - x[1])
    
    
    # Update bot's knowledge about crew at each timestep
    def update_knowledge_crew(self, crew_beep):
        probability_sum = 0
        # if we still have both crews left to find then we will consider crews being present n 2 seperate cells simultaneously and update the belief as a pair
        if self.crew_found == None:

            # dictionary that contains pair of i, j cells as keys and as probability of crew 1 being present in cell i and crew 2 being present in cejj j simultaneously as value
            for key in self.crew_beliefs:

                # now that we have cell pair for crew locations, we will calculate the probability of both individually
                crew_i = key[0]
                crew_j = key[1]

                if self.bot_location in key:
                    self.crew_beliefs[(crew_i, crew_j)] = 0.0 #set belief to 0 for current bot location as we know crew member is not there  
                    continue  # Skip this iteration if bot location is part of key

                distance_to_crew_i = self.get_distance(crew_i)
                distance_to_crew_j = self.get_distance(crew_j)

                # P(beep in bot's cell | crew in cell i)
                crew_i_prob = np.exp(-self.alpha * (distance_to_crew_i - 1))
                # P(beep in bot's cell | crew in cell j)
                crew_j_prob = np.exp(-self.alpha * (distance_to_crew_j - 1))

                # if we hear beep from any of the both crew members then belief is update based on if we recieved beep either from crew i or crew j or both
                if any(crew_beep):
                    likelihood = crew_i_prob + crew_j_prob - crew_i_prob * crew_j_prob
                else:
                    # if we didnt recieve beeps from both crew
                    likelihood = (1-crew_i_prob) * (1-crew_j_prob)
                            
                # Calculate prior probability P(crew_i and crew_j)
                prior_prob = self.crew_beliefs.get((crew_i, crew_j), 0.0)

                new_prob = prior_prob * likelihood
                self.crew_beliefs[(crew_i, crew_j)] = new_prob

                probability_sum += self.crew_beliefs[(crew_i, crew_j)]
            
            # Normalizing 
            normalized_beliefs = {}
            for key, value in self.crew_beliefs.items():
                normalized_beliefs[key] = value / probability_sum
    
            self.crew_beliefs = normalized_beliefs
        
        else:
        # go through each cell and update the probability of crew being present based on the beeps recieved
            for i in range(self.size):
                for j in range(self.size):
                    if self.grid[i][j].is_bot:
                        self.grid[i][j].prob_crew = 0.0
                    elif not self.grid[i][j].is_blocked:
                        distance = self.get_distance((i,j))
                        prob = np.exp(-self.alpha * (distance - 1))
                        # update belief of crew if beef recieved
                        if any(crew_beep):
                            self.grid[i][j].prob_crew *= prob
                        else:
                            self.grid[i][j].prob_crew *= (1 - prob)
                    probability_sum +=self.grid[i][j].prob_crew

            # Normalizing 
            if probability_sum>0:
                for i in range(self.size):
                    for j in range(self.size):
                        if not self.grid[i][j].is_blocked:
                            self.grid[i][j].prob_crew /= probability_sum
            # Append current cell position and beep occurrence to deque

            else:
                self.initialize_prob()
        self.prev_cells.append(self.bot_location)
        self.prev_beeps.append(any(crew_beep))

    def initialize_prob(self):
        n = len(self.open_cells)
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked and not self.grid[i][j].is_bot:
                    self.grid[i][j].prob_crew = 1/(n-1)

    def negate_crew_prob(self):
        # Calculate reduced beep count
        reduced_beep_count = int(len(self.prev_beeps) * 0.5)

        # Make copies of relevant data structures for mock simulation
        prev_cells_copy = list(self.prev_cells)
        prev_beeps_copy = list(self.prev_beeps)[:reduced_beep_count]

        # Perform mock simulation for the last 15 time steps
        for _ in range(15):
            # Update bot's knowledge with reduced beep count
            probability_sum = 0
            for i in range(self.size):
                for j in range(self.size):
                    if self.grid[i][j].is_bot:
                        self.grid[i][j].prob_crew = 0.0
                    elif not self.grid[i][j].is_blocked:
                        distance = self.get_distance((i,j))
                        prob = np.exp(-self.alpha * (distance - 1))
                        if any(prev_beeps_copy):
                            self.grid[i][j].prob_crew *= prob
                        else:
                            self.grid[i][j].prob_crew *= (1 - prob)
                        probability_sum += self.grid[i][j].prob_crew

            # Normalize probabilities
            if probability_sum > 0:
                for i in range(self.size):
                    for j in range(self.size):
                        if not self.grid[i][j].is_blocked:
                            self.grid[i][j].prob_crew /= probability_sum

            # Move bot randomly
            self.move_bot_randomly()

            # Update previous cells and beeps
            self.prev_cells.append(self.bot_location)
            self.prev_beeps.append(any(prev_beeps_copy))
            prev_cells_copy.append(self.bot_location)
            prev_beeps_copy.append(any(prev_beeps_copy))
            prev_cells_copy = prev_cells_copy[-15:]
            prev_beeps_copy = prev_beeps_copy[-reduced_beep_count:]

        # Update crew probability distribution based on mock simulation
        for cell in prev_cells_copy:
            i, j = cell
            if not self.grid[i][j].is_blocked:
                distance = self.get_distance(cell)
                prob = np.exp(-self.alpha * (distance - 1))
                if any(prev_beeps_copy):
                    self.grid[i][j].prob_crew *= prob
                else:
                    self.grid[i][j].prob_crew *= (1 - prob)

        # Normalize crew probability distribution
        probability_sum = sum(cell.prob_crew for row in self.grid for cell in row if not cell.is_blocked)
        if probability_sum > 0:
            for i in range(self.size):
                for j in range(self.size):
                    if not self.grid[i][j].is_blocked:
                        self.grid[i][j].prob_crew /= probability_sum


    def move_bot_randomly(self):
    # Define possible movements
        directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Right, Left, Down, Up

        # Choose a random movement
        movement = random.choice(directions)
        new_location = (self.bot_location[0] + movement[0], self.bot_location[1] + movement[1])

        if 0 <= new_location[0] < self.size and 0 <= new_location[1] < self.size:
            # Update bot's location
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
            self.bot_location = new_location
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
  
     # check if the given cell is in the detection square or not
    def in_detection_square(self, x):
        return abs(x[0] - self.bot_location[0]) <= self.k and abs(x[1] - self.bot_location[1]) <= self.k
    
    # When the aliens move the probability changes based on the number of open cells that the alien can move into
    def calculate_updated_probability(self, pair):
        m, n = pair
        updated_probability=0.0
        m_neighbors = self.open_neighbors.get((m[0], m[1]))
        n_neighbors = self.open_neighbors.get((n[0], n[1]))

        for m_neighbor in m_neighbors:
            for n_neighbor in n_neighbors:
                possible_moves_m = len(self.open_neighbors.get((m_neighbor[0], m_neighbor[1])))
                possible_moves_n = len(self.open_neighbors.get((n_neighbor[0], n_neighbor[1])))
                m_prob = self.grid[m_neighbor[0]][m_neighbor[1]].prob_alien/possible_moves_m
                n_prob = self.grid[n_neighbor[0]][n_neighbor[1]].prob_alien/possible_moves_n
                updated_probability += m_prob*n_prob

        return updated_probability

# Update alien's beliefs
    def update_knowledge_alien(self, Alien_detect):
        # If alien is detected in the detection square
        if any(Alien_detect):
            self.update_probabilities_with_detected_alien()
        else:
            self.update_probabilities_no_alien_detected()

        total_sum = sum(value for value in self.alien_probabilities.values())
        # Normalizing 
        if total_sum != 0 :
            normalized_beliefs = {}
            for key, value in self.alien_probabilities.items():
                normalized_beliefs[key] = value / total_sum
            self.alien_probabilities = normalized_beliefs
        else :
            self.Initialize_alien_beliefs()
        
        sorted_probabilities = sorted(self.alien_probabilities.items(), key=lambda x: x[1], reverse=True)
        # Taking the top 10 coordinate pairs with highest alien probabilities
        top_5_keys = [key for key, _ in sorted_probabilities[:5]]
        # Finding the coordinate with the closest distance to the current cell
        self.pos_with_high_alien_beliefs = self.closest_coordinate(top_5_keys)

    # helper function to get the list of closest coordinates having highest alien belief
    def closest_coordinate(self, coordinates_list):
        closest_coordinates = []
    
        for key in coordinates_list:
            for coord_pair in key:
                distance = self.get_distance(coord_pair)
                # Using a heap to maintain top 5 closest coordinates efficiently based on their distance to the bot
                heapq.heappush(closest_coordinates, (distance, coord_pair))
                if len(closest_coordinates) > 5:
                    heapq.heappop(closest_coordinates)
        
        # Extracting only the coordinates from the heap
        top_5_closest = [coord for _, coord in closest_coordinates]
        return top_5_closest

    def update_probabilities_with_detected_alien(self):
        # Update probabilities based on the presence of aliens in the detection square
        for pair, probability in self.alien_probabilities.items():
            # Check if any of the cells in the pair is in the detection square
            if any(self.in_detection_square(cell) for cell in pair):
                # Update probability using the provided formula
                updated_probability = self.calculate_updated_probability(pair)
                self.alien_probabilities[pair] = updated_probability
            else:
                self.alien_probabilities[pair] = 0.0
        

    def update_probabilities_no_alien_detected(self):
        # Update probabilities when no alien is detected in the detection square
        # Set probabilities to 0 for pairs involving cells in the detection square
        for pair in self.alien_probabilities.keys():
            if any(self.in_detection_square(cell) for cell in pair):
                self.alien_probabilities[pair] = 0.0
            else:
                updated_probability = self.calculate_updated_probability(pair)
                self.alien_probabilities[pair] = updated_probability
        
    
    def find_path(self, src, dest, avoid_alien = True):
        # print("started path planning from", src, "to " , dest)
        visited = set() # Dictionary to store the visited nodes
        parent = {}  # Dictionary to store parent nodes
        # Keep track of the nodes
        fringe = deque([src])
        
        while fringe:
            (row, col) = fringe.popleft()
            # Process the current cell until it reaches the destination
            if (row, col) == dest:
                # Reconstruct the path from dest to src
                path = []
                while (row, col) != src:
                    path.append((row, col))
                    row, col = parent[(row, col)]
                path.reverse()
                return path  # Path found
            
            # Enqueue for unvisited neighbors
            for dr, dc in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                new_row, new_col = row + dr, col + dc
                # Check if the new cell is within the boundaries 0 and length of Ship
                # Check if the new cell is not in alien location and not in visited
                # Check if the new cell is not blocked 
                if 0 <= new_row < self.size and 0 <= new_col < self.size:
                    avoid_condition =  (self.grid[new_row][new_col].prob_alien < self.alien_threshold) or (not avoid_alien and (new_row, new_col) not in self.pos_with_high_alien_beliefs)
                    if (new_row, new_col) not in visited and not self.grid[new_row][new_col].is_blocked and avoid_condition:
                        fringe.append((new_row, new_col))
                        visited.add((new_row, new_col))
                        parent[(new_row, new_col)] = (row, col)

        return None # If path not found
   
    def move_bot_for_1_crew(self, crew_beep):
        # Initialize variables to store the maximum crew belief and the corresponding target cell
        max_crew_belief = 0.0
        target_cells = []
        avoid_alien = True
        best_neighbor = None
        min_alien_neighbor = float('inf')
        max_crew_neighbor = float('-inf')
        neighbors = self.open_neighbors.get((self.bot_location[0], self.bot_location[1]))

         # Prioritize moving towards high-probability crew member cells
        crew_probs = np.array([[cell.prob_crew for cell in row] for row in self.grid])

        # Find the maximum probability value and its indices
        max_prob = crew_probs.max()
        max_indices = np.argwhere(crew_probs == max_prob)

        # Extract the coordinates of cells with the highest probability
        target_cells = [(i, j) for i, j in max_indices]
         # Prioritize moving towards high-probability crew member cells
        for neighbor in neighbors:
            i, j = neighbor
            crew_belief = self.grid[i][j].prob_crew
            alien_belief = self.grid[i][j].prob_alien
            if any(crew_beep):
                if crew_belief > max_crew_neighbor:
                    max_crew_neighbor = crew_belief
                    best_neighbor = neighbor
            else:
                if alien_belief < min_alien_neighbor:
                    min_alien_neighbor = alien_belief
                    best_neighbor = neighbor

        # Choose one of the target cells randomly if there are multiple cells with max crew belief
        if target_cells:
            target_cell = random.choice(target_cells)
        else:
            # If no target cells found (all cells are blocked or have aliens), set target_cell to None
            target_cell = None

        # print(f"bot move to {target_cell} with probability {max_crew_belief}")
        path = self.find_path(self.bot_location, target_cell, avoid_alien)
        if path is not None and len(path)>0:
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
            # Update the bot's location
            self.bot_location = path[0]
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
        else:
            avoid_alien = False
            path = self.find_path(self.bot_location, target_cell, avoid_alien)
            if path is not None and len(path)>0:
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                # Update the bot's location
                self.bot_location = path[0]
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
            else:
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                 # Update the bot's location
                self.bot_location = best_neighbor
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True

    # Movement decision
    def move_bot(self, crew_beep):
        if(self.crew_found != None):
            self.move_bot_for_1_crew(crew_beep)
            return
        # Determine cell most likely to contain crew member
        neighbors = self.open_neighbors.get((self.bot_location[0], self.bot_location[1]))
        avoid_alien = True
        max_crew_belief = 0.0
        best_neighbor = None
        min_alien_neighbor = float('inf')
        max_crew_neighbor = float('-inf')
        target_cell = None
        
        max_crew_belief_cells = max(self.crew_beliefs, key = self.crew_beliefs.get)
        # print('max prob cell', max_crew_belief_cells)
        max_crew_belief = self.crew_beliefs[max_crew_belief_cells]
        
        for neighbor in neighbors:
            i, j = neighbor
            crew_belief = self.grid[i][j].prob_crew
            alien_belief = self.grid[i][j].prob_alien
            if any(crew_beep):
                if crew_belief > max_crew_neighbor:
                    max_crew_neighbor = crew_belief
                    best_neighbor = neighbor
            else:
                if alien_belief < min_alien_neighbor:
                    min_alien_neighbor = alien_belief
                    best_neighbor = neighbor
                        
        crew1_cell = max_crew_belief_cells[0]
        crew2_cell = max_crew_belief_cells[1]

        crew1_distance = self.get_distance(crew1_cell)
        crew2_distance = self.get_distance(crew2_cell)

        target_cell = crew1_cell if crew1_distance < crew2_distance and self.bot_location != crew1_cell else (crew2_cell if self.bot_location != crew2_cell else None)

        # print("bot move to ", target_cell, "with probability",max_crew_belief)
        path = self.find_path(self.bot_location, target_cell)

        if path is not None and len(path)>0:
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
            self.bot_location = path[0]
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
        else:
            avoid_alien = False
            path = self.find_path(self.bot_location, target_cell, avoid_alien)
            if path is not None and len(path)>0:
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                # Update the bot's location
                self.bot_location = path[0]
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
            else:
                avoid_alien = True
                # First attempt failed, try the other crew member's cell as the target
                other_target_cell = crew2_cell if target_cell == crew1_cell else crew1_cell
                path = self.find_path(self.bot_location, other_target_cell, avoid_alien)
                if path is not None and len(path) > 0:
                    self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                    self.bot_location = path[0]
                    self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
                else:
                    avoid_alien = False
                    path = self.find_path(self.bot_location, other_target_cell, avoid_alien)
                    if path is not None and len(path) > 0:
                        self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                        self.bot_location = path[0]
                        self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
                    else:
                        self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                        # Update the bot's location
                        self.bot_location = best_neighbor
                        self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True            


    # Handling alien movement
    def handle_alien_movement(self):
        new_alien_positions = []
        for i in range(len(self.all_alien_locations)):
            x, y = self.all_alien_locations[i]
            directions = [(0,1),(0,-1),(-1,0),(1,0)]
            random.shuffle(directions)
            possible_moves = [
            (nx, ny) 
            for dx, dy in directions
            if 0 <= (nx := x + dx) < self.size and 0 <= (ny := y + dy) < self.size and not self.grid[nx][ny].is_blocked and not self.grid[nx][ny].is_Alien
            ]
            if(possible_moves):
                new_position = random.choice(possible_moves)
                new_alien_positions.append(new_position)
                ax, ay = new_position
                self.grid[ax][ay].is_Alien = True
                self.grid[x][y].is_Alien = False
        
        self.all_alien_locations = new_alien_positions
        
    # main function
    def bot_function(self):
        steps = 1000
        self.Initialize_crew_beliefs()
        self.Initialize_alien_beliefs()
        num_of_crew_reached = 0
        for step in range(steps):
            # print(step)
            # print("self.crew_locations self.bot_location self.alien_locations", self.crew_locations, self.bot_location, self.all_alien_locations)
            # Check if the bot reaches the crew
            crew_beep = []
            Alien_detect = []
            for crew_location in self.crew_locations:
                crew_beep.append(self.check_beep_for_crew(crew_location))
            for alien in self.all_alien_locations:
                Alien_detect.append(True if self.in_detection_square(alien) else False)
            # s = time.time()
            self.update_knowledge_crew(crew_beep)
            # print('time taken for crew belief',(time.time() - s))
            # s = time.time()
            self.update_knowledge_alien(Alien_detect)
            # print('time taken for alien belief',(time.time() - s))
            # s = time.time()
            self.move_bot(crew_beep)
            # print('bot is moving people',(time.time() - s))
            if self.bot_location in self.crew_locations:
                num_of_crew_reached += 1
                self.crew_found = self.bot_location
                if num_of_crew_reached == 2:
                    # print("Congratulations! Bot reached the crew in steps:", step)
                    return True, step, num_of_crew_reached
                else:
                    # print("Congratz, CREW1 Reached")
                    self.grid[self.bot_location[0]][self.bot_location[1]].is_crew =  False
                    self.crew_locations.remove(self.bot_location)
                    crew_prob_matrix = np.zeros((self.size, self.size))

                    # Update each cell's probability to be the sum of probabilities of all combinations of crew members
                    for i in range(self.size):
                        for j in range(self.size):
                            total_prob = 0
                            for (x, y), prob in self.crew_beliefs.items():
                                if (x == i and y == j) or (x == j and y == i):
                                    total_prob += prob
                            crew_prob_matrix[i][j] = total_prob

                    for i in range(self.size):
                        for j in range(self.size):
                            self.grid[i][j].prob_crew = crew_prob_matrix[i][j]

                    self.negate_crew_prob()

            
            # Check if the bot is caught by an alien
            if self.bot_location in self.all_alien_locations:
                # print("Game over! Bot caught by an alien in steps:", step)
                return False, step, num_of_crew_reached
            
            for alien in self.all_alien_locations:
                Alien_detect.append(True if self.in_detection_square(alien) else False)
            
            self.update_knowledge_alien(Alien_detect)
            self.handle_alien_movement()
            # visualize_grid(self.grid)
            # Check if the bot reaches the crew
            if self.bot_location in self.all_alien_locations:
                # print("Game over! Bot caught by an alien in steps:", step)
                return False, step, num_of_crew_reached
            
        # print("Game over! Bot couldn't reach the crew in steps:", step)
        return False, step, num_of_crew_reached

In [ ]:
class Bot8:
    def __init__(self, D, k, alpha, ship):
        self.size = D
        self.ship = ship
        self.k = k
        self.grid = self.ship.grid
        self.bot_location = self.ship.bot_location
        self.all_alien_locations = self.ship.all_alien_locations
        self.crew_locations = self.ship.crew_locations
        self.alpha = alpha
        self.crew_beliefs = {}
        self.crew_found = None
        self.alien_probabilities = {}  # Dictionary to store pairwise alien probabilities
        self.pos_with_high_alien_beliefs = []
        self.open_cells = self.ship.open_cells
        self.open_neighbors = self.ship.open_neighbors
        self.alien_threshold = 0.01
        self.prev_cells = deque(maxlen=15)
        self.prev_beeps = deque(maxlen=15)

    def Initialize_crew_beliefs(self):
        num_open_cells = len(self.open_cells)
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked and not self.grid[i][j].is_bot:
                    for m in range(self.size):
                        for n in range(self.size):
                            if not self.grid[m][n].is_blocked and not self.grid[i][j].is_bot and (i!=m and j!=n):
                                key = tuple(sorted(((i, j), (m, n))))  # Sort the tuple to ensure consistent order key = ((i, j), (m, n))
                                self.crew_beliefs[key] = 1/(num_open_cells-1)*1/(num_open_cells-1)

    def Initialize_alien_beliefs(self):
        k = 0
        # check for number of open cells in detection square 
        for i in range(self.size):
            for j in range(self.size):
                if self.in_detection_square((i,j)) and not self.grid[i][j].is_blocked:
                    k +=1
        num_open_cells = len(self.open_cells)-k        
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked:
                    for m in range(self.size):
                        for n in range(self.size):
                            if not self.grid[m][n].is_blocked and (i != m or j != n):
                                key = tuple(sorted(((i, j), (m, n))))  # Sort the tuple to ensure consistent order
                                self.alien_probabilities[key] = 1/(num_open_cells-1)*1/(num_open_cells-1)
    
    def check_beep_for_crew(self, crew_position):
        # Calculate Manhattan distance between bot and crew member
        distance = abs(self.bot_location[0] - crew_position[0]) + abs(self.bot_location[1] - crew_position[1])
        # Calculate probability of hearing the beep
        beep_prob = np.exp(-self.alpha * (distance - 1))
        rand_num = random.random()
        if rand_num <= beep_prob:
            return True  # Bot hears the beep
        else:
            return False  # Bot does not hear the beep

    def check_for_alien_beep(self):
        for i in range(-self.k, self.k+1):
            for j in range(-self.k, self.k+1):
                x = self.bot_location[0]+i
                y = self.bot_location[1]+j
                if 0<=x<self.size and 0<=y<self.size and self.grid[x][y].is_Alien:
                    return True
        return False 

    def get_distance(self, x):
        return  abs(self.bot_location[0] - x[0]) + abs(self.bot_location[1] - x[1])
    
    
    # Update bot's knowledge about crew at each timestep
    def update_knowledge_crew(self, crew_beep):
        probability_sum = 0
        # if we still have both crews left to find then we will consider crews being present n 2 seperate cells simultaneously and update the belief as a pair
        if self.crew_found == None:

            # dictionary that contains pair of i, j cells as keys and as probability of crew 1 being present in cell i and crew 2 being present in cejj j simultaneously as value
            for key in self.crew_beliefs:

                # now that we have cell pair for crew locations, we will calculate the probability of both individually
                crew_i = key[0]
                crew_j = key[1]

                if self.bot_location in key:
                    self.crew_beliefs[(crew_i, crew_j)] = 0.0 #set belief to 0 for current bot location as we know crew member is not there  
                    continue  # Skip this iteration if bot location is part of key

                distance_to_crew_i = self.get_distance(crew_i)
                distance_to_crew_j = self.get_distance(crew_j)

                # P(beep in bot's cell | crew in cell i)
                crew_i_prob = np.exp(-self.alpha * (distance_to_crew_i - 1))
                # P(beep in bot's cell | crew in cell j)
                crew_j_prob = np.exp(-self.alpha * (distance_to_crew_j - 1))

                # if we hear beep from any of the both crew members then belief is update based on if we recieved beep either from crew i or crew j or both
                if any(crew_beep):
                    likelihood = crew_i_prob + crew_j_prob - crew_i_prob * crew_j_prob
                else:
                    # if we didnt recieve beeps from both crew
                    likelihood = (1-crew_i_prob) * (1-crew_j_prob)
                            
                # Calculate prior probability P(crew_i and crew_j)
                prior_prob = self.crew_beliefs.get((crew_i, crew_j), 0.0)

                new_prob = prior_prob * likelihood
                self.crew_beliefs[(crew_i, crew_j)] = new_prob

                probability_sum += self.crew_beliefs[(crew_i, crew_j)]
            
            # Normalizing 
            normalized_beliefs = {}
            for key, value in self.crew_beliefs.items():
                normalized_beliefs[key] = value / probability_sum
    
            self.crew_beliefs = normalized_beliefs
        
        else:
        # go through each cell and update the probability of crew being present based on the beeps recieved
            for i in range(self.size):
                for j in range(self.size):
                    if self.grid[i][j].is_bot:
                        self.grid[i][j].prob_crew = 0.0
                    elif not self.grid[i][j].is_blocked:
                        distance = self.get_distance((i,j))
                        prob = np.exp(-self.alpha * (distance - 1))
                        # update belief of crew if beef recieved
                        if any(crew_beep):
                            self.grid[i][j].prob_crew *= prob
                        else:
                            self.grid[i][j].prob_crew *= (1 - prob)
                    probability_sum +=self.grid[i][j].prob_crew

            # Normalizing 
            if probability_sum>0:
                for i in range(self.size):
                    for j in range(self.size):
                        if not self.grid[i][j].is_blocked:
                            self.grid[i][j].prob_crew /= probability_sum
            # Append current cell position and beep occurrence to deque

            else:
                self.initialize_prob()
        self.prev_cells.append(self.bot_location)
        self.prev_beeps.append(any(crew_beep))

    def initialize_prob(self):
        n = len(self.open_cells)
        for i in range(self.size):
            for j in range(self.size):
                if not self.grid[i][j].is_blocked and not self.grid[i][j].is_bot:
                    self.grid[i][j].prob_crew = 1/(n-1)

    def negate_crew_prob(self):
        # Calculate reduced beep count
        reduced_beep_count = int(len(self.prev_beeps) * 0.5)

        # Make copies of relevant data structures for mock simulation
        prev_cells_copy = list(self.prev_cells)
        prev_beeps_copy = list(self.prev_beeps)[:reduced_beep_count]

        # Perform mock simulation for the last 15 time steps
        for _ in range(15):
            # Update bot's knowledge with reduced beep count
            probability_sum = 0
            for i in range(self.size):
                for j in range(self.size):
                    if self.grid[i][j].is_bot:
                        self.grid[i][j].prob_crew = 0.0
                    elif not self.grid[i][j].is_blocked:
                        distance = self.get_distance((i,j))
                        prob = np.exp(-self.alpha * (distance - 1))
                        if any(prev_beeps_copy):
                            self.grid[i][j].prob_crew *= prob
                        else:
                            self.grid[i][j].prob_crew *= (1 - prob)
                        probability_sum += self.grid[i][j].prob_crew

            # Normalize probabilities
            if probability_sum > 0:
                for i in range(self.size):
                    for j in range(self.size):
                        if not self.grid[i][j].is_blocked:
                            self.grid[i][j].prob_crew /= probability_sum

            # Move bot randomly
            self.move_bot_randomly()

            # Update previous cells and beeps
            self.prev_cells.append(self.bot_location)
            self.prev_beeps.append(any(prev_beeps_copy))
            prev_cells_copy.append(self.bot_location)
            prev_beeps_copy.append(any(prev_beeps_copy))
            prev_cells_copy = prev_cells_copy[-15:]
            prev_beeps_copy = prev_beeps_copy[-reduced_beep_count:]

        # Update crew probability distribution based on mock simulation
        for cell in prev_cells_copy:
            i, j = cell
            if not self.grid[i][j].is_blocked:
                distance = self.get_distance(cell)
                prob = np.exp(-self.alpha * (distance - 1))
                if any(prev_beeps_copy):
                    self.grid[i][j].prob_crew *= prob
                else:
                    self.grid[i][j].prob_crew *= (1 - prob)

        # Normalize crew probability distribution
        probability_sum = sum(cell.prob_crew for row in self.grid for cell in row if not cell.is_blocked)
        if probability_sum > 0:
            for i in range(self.size):
                for j in range(self.size):
                    if not self.grid[i][j].is_blocked:
                        self.grid[i][j].prob_crew /= probability_sum


    def move_bot_randomly(self):
    # Define possible movements
        directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Right, Left, Down, Up

        # Choose a random movement
        movement = random.choice(directions)
        new_location = (self.bot_location[0] + movement[0], self.bot_location[1] + movement[1])

        if 0 <= new_location[0] < self.size and 0 <= new_location[1] < self.size:
            # Update bot's location
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
            self.bot_location = new_location
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
  
     # check if the given cell is in the detection square or not
    def in_detection_square(self, x):
        return abs(x[0] - self.bot_location[0]) <= self.k and abs(x[1] - self.bot_location[1]) <= self.k
    
    # When the aliens move the probability changes based on the number of open cells that the alien can move into
    def calculate_updated_probability(self, pair):
        m, n = pair
        updated_probability=0.0
        m_neighbors = self.open_neighbors.get((m[0], m[1]))
        n_neighbors = self.open_neighbors.get((n[0], n[1]))

        for m_neighbor in m_neighbors:
            for n_neighbor in n_neighbors:
                possible_moves_m = len(self.open_neighbors.get((m_neighbor[0], m_neighbor[1])))
                possible_moves_n = len(self.open_neighbors.get((n_neighbor[0], n_neighbor[1])))
                m_prob = self.grid[m_neighbor[0]][m_neighbor[1]].prob_alien/possible_moves_m
                n_prob = self.grid[n_neighbor[0]][n_neighbor[1]].prob_alien/possible_moves_n
                updated_probability += m_prob*n_prob

        return updated_probability

    def get_manhattan_distance(self, x, y):
        return  abs(y[0] - x[0]) + abs(y[1] - x[1])
# Update alien's beliefs
    def update_knowledge_alien(self, Alien_detect):
        # If alien is detected in the detection square
        if any(Alien_detect):
            self.update_probabilities_with_detected_alien()
        else:
            self.update_probabilities_no_alien_detected()
        
        for key, value in self.alien_probabilities.items():
            x,y = key
            dist = self.get_manhattan_distance(x,y)
            self.alien_probabilities[key] = value * (1/dist)
        
        total_sum = sum(value for value in self.alien_probabilities.values())
        # Normalizing 
        if total_sum != 0 :
            normalized_beliefs = {}
            for key, value in self.alien_probabilities.items():
                normalized_beliefs[key] = value / total_sum
            self.alien_probabilities = normalized_beliefs
        else :
            self.Initialize_alien_beliefs()
        
        sorted_probabilities = sorted(self.alien_probabilities.items(), key=lambda x: x[1], reverse=True)
        # Taking the top 10 coordinate pairs with highest alien probabilities
        top_5_keys = [key for key, _ in sorted_probabilities[:5]]
        # Finding the coordinate with the closest distance to the current cell
        self.pos_with_high_alien_beliefs = self.closest_coordinate(top_5_keys)

    # helper function to get the list of closest coordinates having highest alien belief
    def closest_coordinate(self, coordinates_list):
        closest_coordinates = []
    
        for key in coordinates_list:
            for coord_pair in key:
                distance = self.get_distance(coord_pair)
                # Using a heap to maintain top 5 closest coordinates efficiently based on their distance to the bot
                heapq.heappush(closest_coordinates, (distance, coord_pair))
                if len(closest_coordinates) > 5:
                    heapq.heappop(closest_coordinates)
        
        # Extracting only the coordinates from the heap
        top_5_closest = [coord for _, coord in closest_coordinates]
        return top_5_closest

    def update_probabilities_with_detected_alien(self):
        # Update probabilities based on the presence of aliens in the detection square
        for pair, probability in self.alien_probabilities.items():
            # Check if any of the cells in the pair is in the detection square
            if any(self.in_detection_square(cell) for cell in pair):
                # Update probability using the provided formula
                updated_probability = self.calculate_updated_probability(pair)
                self.alien_probabilities[pair] = updated_probability
            else:
                self.alien_probabilities[pair] = 0.0
        

    def update_probabilities_no_alien_detected(self):
        # Update probabilities when no alien is detected in the detection square
        # Set probabilities to 0 for pairs involving cells in the detection square
        for pair in self.alien_probabilities.keys():
            if any(self.in_detection_square(cell) for cell in pair):
                self.alien_probabilities[pair] = 0.0
            else:
                updated_probability = self.calculate_updated_probability(pair)
                self.alien_probabilities[pair] = updated_probability
        
    
    def find_path(self, src, dest, avoid_alien = True):
        # print("started path planning from", src, "to " , dest)
        visited = set() # Dictionary to store the visited nodes
        parent = {}  # Dictionary to store parent nodes
        # Keep track of the nodes
        fringe = deque([src])
        
        while fringe:
            (row, col) = fringe.popleft()
            # Process the current cell until it reaches the destination
            if (row, col) == dest:
                # Reconstruct the path from dest to src
                path = []
                while (row, col) != src:
                    path.append((row, col))
                    row, col = parent[(row, col)]
                path.reverse()
                return path  # Path found
            
            # Enqueue for unvisited neighbors
            for dr, dc in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                new_row, new_col = row + dr, col + dc
                # Check if the new cell is within the boundaries 0 and length of Ship
                # Check if the new cell is not in alien location and not in visited
                # Check if the new cell is not blocked 
                if 0 <= new_row < self.size and 0 <= new_col < self.size:
                    avoid_condition =  (self.grid[new_row][new_col].prob_alien < self.alien_threshold) or (not avoid_alien and (new_row, new_col) not in self.pos_with_high_alien_beliefs)
                    if (new_row, new_col) not in visited and not self.grid[new_row][new_col].is_blocked and avoid_condition:
                        fringe.append((new_row, new_col))
                        visited.add((new_row, new_col))
                        parent[(new_row, new_col)] = (row, col)

        return None # If path not found
   
    def move_bot_for_1_crew(self, crew_beep):
        # Initialize variables to store the maximum crew belief and the corresponding target cell
        max_crew_belief = 0.0
        target_cells = []
        avoid_alien = True
        best_neighbor = None
        min_alien_neighbor = float('inf')
        max_crew_neighbor = float('-inf')
        neighbors = self.open_neighbors.get((self.bot_location[0], self.bot_location[1]))

         # Prioritize moving towards high-probability crew member cells
        crew_probs = np.array([[cell.prob_crew for cell in row] for row in self.grid])

        # Find the maximum probability value and its indices
        max_prob = crew_probs.max()
        max_indices = np.argwhere(crew_probs == max_prob)

        # Extract the coordinates of cells with the highest probability
        target_cells = [(i, j) for i, j in max_indices]
         # Prioritize moving towards high-probability crew member cells
        for neighbor in neighbors:
            i, j = neighbor
            crew_belief = self.grid[i][j].prob_crew
            alien_belief = self.grid[i][j].prob_alien
            if any(crew_beep):
                if crew_belief > max_crew_neighbor:
                    max_crew_neighbor = crew_belief
                    best_neighbor = neighbor
            else:
                if alien_belief < min_alien_neighbor:
                    min_alien_neighbor = alien_belief
                    best_neighbor = neighbor

        # Choose one of the target cells randomly if there are multiple cells with max crew belief
        if target_cells:
            target_cell = random.choice(target_cells)
        else:
            # If no target cells found (all cells are blocked or have aliens), set target_cell to None
            target_cell = None

        # print(f"bot move to {target_cell} with probability {max_crew_belief}")
        path = self.find_path(self.bot_location, target_cell, avoid_alien)
        if path is not None and len(path)>0:
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
            # Update the bot's location
            self.bot_location = path[0]
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
        else:
            avoid_alien = False
            path = self.find_path(self.bot_location, target_cell, avoid_alien)
            if path is not None and len(path)>0:
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                # Update the bot's location
                self.bot_location = path[0]
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
            else:
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                 # Update the bot's location
                self.bot_location = best_neighbor
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True

    # Movement decision
    def move_bot(self, crew_beep):
        if(self.crew_found != None):
            self.move_bot_for_1_crew(crew_beep)
            return
        # Determine cell most likely to contain crew member
        neighbors = self.open_neighbors.get((self.bot_location[0], self.bot_location[1]))
        avoid_alien = True
        max_crew_belief = 0.0
        best_neighbor = None
        min_alien_neighbor = float('inf')
        max_crew_neighbor = float('-inf')
        target_cell = None
        
        max_crew_belief_cells = max(self.crew_beliefs, key = self.crew_beliefs.get)
        # print('max prob cell', max_crew_belief_cells)
        max_crew_belief = self.crew_beliefs[max_crew_belief_cells]
        
        for neighbor in neighbors:
            i, j = neighbor
            crew_belief = self.grid[i][j].prob_crew
            alien_belief = self.grid[i][j].prob_alien
            if any(crew_beep):
                if crew_belief > max_crew_neighbor:
                    max_crew_neighbor = crew_belief
                    best_neighbor = neighbor
            else:
                if alien_belief < min_alien_neighbor:
                    min_alien_neighbor = alien_belief
                    best_neighbor = neighbor
                        
        crew1_cell = max_crew_belief_cells[0]
        crew2_cell = max_crew_belief_cells[1]

        crew1_distance = self.get_distance(crew1_cell)
        crew2_distance = self.get_distance(crew2_cell)

        target_cell = crew1_cell if crew1_distance < crew2_distance and self.bot_location != crew1_cell else (crew2_cell if self.bot_location != crew2_cell else None)

        # print("bot move to ", target_cell, "with probability",max_crew_belief)
        path = self.find_path(self.bot_location, target_cell)

        if path is not None and len(path)>0:
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
            self.bot_location = path[0]
            self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
        else:
            avoid_alien = False
            path = self.find_path(self.bot_location, target_cell, avoid_alien)
            if path is not None and len(path)>0:
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                # Update the bot's location
                self.bot_location = path[0]
                self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
            else:
                avoid_alien = True
                # First attempt failed, try the other crew member's cell as the target
                other_target_cell = crew2_cell if target_cell == crew1_cell else crew1_cell
                path = self.find_path(self.bot_location, other_target_cell, avoid_alien)
                if path is not None and len(path) > 0:
                    self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                    self.bot_location = path[0]
                    self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
                else:
                    avoid_alien = False
                    path = self.find_path(self.bot_location, other_target_cell, avoid_alien)
                    if path is not None and len(path) > 0:
                        self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                        self.bot_location = path[0]
                        self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True
                    else:
                        self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = False
                        # Update the bot's location
                        self.bot_location = best_neighbor
                        self.grid[self.bot_location[0]][self.bot_location[1]].is_bot = True            


    # Handling alien movement
    def handle_alien_movement(self):
        new_alien_positions = []
        for i in range(len(self.all_alien_locations)):
            x, y = self.all_alien_locations[i]
            directions = [(0,1),(0,-1),(-1,0),(1,0)]
            random.shuffle(directions)
            possible_moves = [
            (nx, ny) 
            for dx, dy in directions
            if 0 <= (nx := x + dx) < self.size and 0 <= (ny := y + dy) < self.size and not self.grid[nx][ny].is_blocked and not self.grid[nx][ny].is_Alien
            ]
            if(possible_moves):
                new_position = random.choice(possible_moves)
                new_alien_positions.append(new_position)
                ax, ay = new_position
                self.grid[ax][ay].is_Alien = True
                self.grid[x][y].is_Alien = False
        
        self.all_alien_locations = new_alien_positions
        
    # main function
    def bot_function(self):
        steps = 1000
        self.Initialize_crew_beliefs()
        self.Initialize_alien_beliefs()
        num_of_crew_reached = 0
        for step in range(steps):
            # print(step)
            # print("self.crew_locations self.bot_location self.alien_locations", self.crew_locations, self.bot_location, self.all_alien_locations)
            # Check if the bot reaches the crew
            crew_beep = []
            Alien_detect = []
            for crew_location in self.crew_locations:
                crew_beep.append(self.check_beep_for_crew(crew_location))
            for alien in self.all_alien_locations:
                Alien_detect.append(True if self.in_detection_square(alien) else False)
            # s = time.time()
            self.update_knowledge_crew(crew_beep)
            # print('time taken for crew belief',(time.time() - s))
            # s = time.time()
            self.update_knowledge_alien(Alien_detect)
            # print('time taken for alien belief',(time.time() - s))
            # s = time.time()
            self.move_bot(crew_beep)
            # print('bot is moving people',(time.time() - s))
            if self.bot_location in self.crew_locations:
                num_of_crew_reached += 1
                self.crew_found = self.bot_location
                if num_of_crew_reached == 2:
                    # print("Congratulations! Bot reached the crew in steps:", step)
                    return True, step, num_of_crew_reached
                else:
                    # print("Congratz, CREW1 Reached")
                    self.grid[self.bot_location[0]][self.bot_location[1]].is_crew =  False
                    self.crew_locations.remove(self.bot_location)
                    crew_prob_matrix = np.zeros((self.size, self.size))

                    # Update each cell's probability to be the sum of probabilities of all combinations of crew members
                    for i in range(self.size):
                        for j in range(self.size):
                            total_prob = 0
                            for (x, y), prob in self.crew_beliefs.items():
                                if (x == i and y == j) or (x == j and y == i):
                                    total_prob += prob
                            crew_prob_matrix[i][j] = total_prob

                    for i in range(self.size):
                        for j in range(self.size):
                            self.grid[i][j].prob_crew = crew_prob_matrix[i][j]

                    self.negate_crew_prob()

            
            # Check if the bot is caught by an alien
            if self.bot_location in self.all_alien_locations:
                # print("Game over! Bot caught by an alien in steps:", step)
                return False, step, num_of_crew_reached
            
            for alien in self.all_alien_locations:
                Alien_detect.append(True if self.in_detection_square(alien) else False)
            
            self.update_knowledge_alien(Alien_detect)
            self.handle_alien_movement()
            # visualize_grid(self.grid)
            # Check if the bot reaches the crew
            if self.bot_location in self.all_alien_locations:
                # print("Game over! Bot caught by an alien in steps:", step)
                return False, step, num_of_crew_reached
            
        # print("Game over! Bot couldn't reach the crew in steps:", step)
        return False, step, num_of_crew_reached

In [ ]:
def visualize_grid(grid):
    # Extract grid size
    rows = len(grid)
    cols = len(grid[0])

    red = [1., 0., 0.]
    pink = [1., 0.5, 0.]
    blue = [0., 0., 1.]
    green = [0., 1., 0.]
    yellow = [1., 1., 0.]
    white = [1., 1., 1.]
    black = [0., 0., 0.]
    grid_img = []
    open_cells = []
    for i in range(rows):
        for j in range(cols):
            if not grid[i][j].is_blocked:
                open_cells.append((i,j))
    beliefs_flat = [grid[oc[0]][oc[1]].prob_crew for oc in open_cells]
    max_belief = max(beliefs_flat)
    print(f"Max Belief: {max_belief}")
    beliefs2_flat = [grid[oc[0]][oc[1]].prob_alien for oc in open_cells]
    max_belief2 = max(beliefs2_flat)
    print(f"Max Belief: {max_belief2}")
    for j in range(rows):
        grid_img.append([])
        for i in range(cols):
            if grid[i][j].is_crew:
                grid_img[-1].append(green)
            elif grid[i][j].is_bot:
                grid_img[-1].append(yellow)
            elif grid[i][j].is_Alien:
                grid_img[-1].append(red)
            elif not grid[i][j].is_blocked:
                grid_img[-1].append([c*grid[i][j].prob_crew/max_belief for c in blue])
                if grid[i][j].prob_crew < 0:
                    print("TOO LOW")
            elif not grid[i][j].is_blocked:
                grid_img[-1].append([c*grid[i][j].prob_alien/max_belief2 for c in pink])
                if grid[i][j].prob_alien < 0:
                    print("TOO LOW")
            else:
                grid_img[-1].append(white)
    # Create a new figure
    plt.figure()

    # Plot obstacles (black cells)
    plt.imshow(grid_img, interpolation='nearest')
    plt.title('Grid Visualization')

    # Show grid
    plt.show()

In [ ]:
def manage_results(bot_behavior):
    success_count = []    # Success count
    k = 2   # Detection square
    D = 35
    crew_members_saved = []
    avg_total_steps = []  # List to store the average total number of steps for successful simulations
    alpha = 0.1
    alpha_values=[]
    while alpha<=0.4:
        alpha_values.append(k)
        temp_crew_list = []  #List to store the result of simulation 1. 'Captain' saved - SUCCESS!! 2. 'Steps' exhausted - Survived!! 3. 'Aliens' captured bot - Dead :(
        total_steps_list = []  # List to store the total number of steps for each simulation
        temp_success_list = []
        ship = Ship_Layout(D)
        ship.create_ship()
        if bot_behavior in ('bot1','bot2'):
          num_aliens = 1
          num_crew = 1
        elif bot_behavior in ('bot3','bot4','bot5'):
          num_aliens = 1
          num_crew = 2
        elif bot_behavior in ('bot6','bot7','bot8'):
          num_aliens = 2
          num_crew = 2
        # Run simulation 50 times for each num of aliens
        for _ in range(1):  
            ship_work = copy.deepcopy(ship)
            ship_work.create_ship_and_place_crew(num_aliens, num_crew, k)
            if bot_behavior == 'bot1':
                bot_instance = Bot1(D, k, alpha, ship_work)
            elif bot_behavior == 'bot2':
                bot_instance = Bot2(D, k, alpha, ship_work)
            elif bot_behavior == 'bot3':
                bot_instance = Bot3(D, k, alpha, ship_work)
            elif bot_behavior == 'bot4':
                bot_instance = Bot4(D, k, alpha, ship_work)
            elif bot_behavior == 'bot5':
                bot_instance = Bot5(D, k, alpha, ship_work)
            elif bot_behavior == 'bot6':
                bot_instance = Bot6(D, k, alpha, ship_work)
            elif bot_behavior == 'bot7':
                bot_instance = Bot7(D, k, alpha, ship_work)
            elif bot_behavior == 'bot8':
                bot_instance = Bot8(D, k, alpha, ship_work)
            success, steps, crew = bot_instance.bot_function()
            # Add the result of simulation in the list
            temp_crew_list.append(crew)
            # Total no. of steps bots survived
            total_steps_list.append(steps if success else 0)
            temp_success_list.append(1 if success else 0)

        alpha += 0.05
        avg_steps = 0
        # Calculate the average total number of steps only for successful simulations
        if total_steps_list:
        # Calculate the average total number of steps for this value of k
            avg_steps = sum(total_steps_list) / len(total_steps_list)
        else:
            avg_steps = 0
        avg_total_steps.append(avg_steps)

        avg_crew_saved = 0
        # Calculate the average total number of steps only for successful simulations
        if temp_crew_list:
        # Calculate the average total number of steps for this value of k
            avg_crew_saved = sum(temp_crew_list) / len(temp_crew_list)
        else:
            avg_crew_saved = 0
        crew_members_saved.append(avg_crew_saved)

        total_success = 0
        if temp_success_list:
        # Calculate the average total number of steps for this value of k
            total_success = sum(temp_success_list) / len(temp_success_list)
        else:
            total_success = 0
        success_count.append(total_success)

    return success_count, crew_members_saved, avg_total_steps, alpha_values

                
def working_of_all_bots():
    
    bot1_data = manage_results('bot1')
    bot2_data = manage_results('bot2')
    bot3_data = manage_results('bot3')
    bot4_data = manage_results('bot4')
    bot5_data = manage_results('bot5')
    bot6_data = manage_results('bot6')
    bot7_data = manage_results('bot7')
    bot8_data = manage_results('bot8')
    
    print('Success Rate: ', bot1_data[0]) #Success Rate for Bot 1
    print('Crew Members Saved ', bot1_data[1]) # Average crew members saved
    print('Avg no. of steps Bot Survived:', bot1_data[2]) #Avg no. of steps bot survived
    
    print('Success Rate: ', bot2_data[0]) #Success Rate for Bot 1
    print('Crew Members Saved ', bot2_data[1]) # Average crew members saved
    print('Avg no. of steps Bot Survived:', bot2_data[2]) #Avg no. of steps bot survived

    print('Success Rate: ', bot3_data[0]) #Success Rate for Bot 1
    print('Crew Members Saved ', bot3_data[1]) # Average crew members saved
    print('Avg no. of steps Bot Survived:', bot3_data[2]) #Avg no. of steps bot survived

    print('Success Rate: ', bot4_data[0]) #Success Rate for Bot 1
    print('Crew Members Saved ', bot4_data[1]) # Average crew members saved
    print('Avg no. of steps Bot Survived:', bot4_data[2]) #Avg no. of steps bot survived

    print('Success Rate: ', bot5_data[0]) #Success Rate for Bot 1
    print('Crew Members Saved ', bot5_data[1]) # Average crew members saved
    print('Avg no. of steps Bot Survived:', bot5_data[2]) #Avg no. of steps bot survived

    print('Success Rate: ', bot6_data[0]) #Success Rate for Bot 1
    print('Crew Members Saved ', bot6_data[1]) # Average crew members saved
    print('Avg no. of steps Bot Survived:', bot6_data[2]) #Avg no. of steps bot survived

    print('Success Rate: ', bot7_data[0]) #Success Rate for Bot 1
    print('Crew Members Saved ', bot7_data[1]) # Average crew members saved
    print('Avg no. of steps Bot Survived:', bot7_data[2]) #Avg no. of steps bot survived


    print('Success Rate: ', bot8_data[0]) #Success Rate for Bot 1
    print('Crew Members Saved ', bot8_data[1]) # Average crew members saved
    print('Avg no. of steps Bot Survived:', bot8_data[2]) #Avg no. of steps bot survived
    
    # plot_graphs(bot1_data)


In [ ]:
# Entry point of the script
if __name__ == "__main__":
   working_of_all_bots()